In [ ]:
# 1_osm_seed.py
import requests, pandas as pd

OVERPASS = "https://overpass-api.de/api/interpreter"

# 도봉구 경계 안의 카페/공원/전망/피크닉 후보
query = r"""
[out:json][timeout:30];
relation["name"="도봉구"]["admin_level"="6"]["boundary"="administrative"];
map_to_area->.searchArea;
(
  node["amenity"="cafe"](area.searchArea);
  node["amenity"="library"](area.searchArea);
  node["tourism"="viewpoint"](area.searchArea);
  way["leisure"="park"](area.searchArea);
  way["leisure"="track"](area.searchArea);
  way["leisure"="garden"](area.searchArea);
);
out center tags;
"""

resp = requests.post(OVERPASS, data={'data': query})
resp.raise_for_status()
data = resp.json()["elements"]

rows=[]
for e in data:
    tags = e.get("tags", {})
    lat = e.get("lat") or e.get("center",{}).get("lat")
    lon = e.get("lon") or e.get("center",{}).get("lon")
    name = tags.get("name")
    if not (name and lat and lon):
        continue
    cat = (tags.get("amenity") or tags.get("tourism") or tags.get("leisure") or "").upper()
    rows.append({"name": name, "category": cat, "lat": lat, "lon": lon})

df = pd.DataFrame(rows).drop_duplicates(subset=["name","lat","lon"])
#df.to_csv("dobong_seed.csv", index=False, encoding="utf-8-sig")
print("도봉구 POI 개수:", len(df))


도봉구 POI 개수: 62


In [6]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

"""
도봉구 '느좋(Good Vibe)' & '숨은 핫플(Hidden Hot)' 추출기
- 키 없이도 OSM(Overpass)만으로 실행 가능
- Kakao Local API 키를 넣으면(선택) 결과가 더 많고 디테일해짐
- 출력: places_master.csv, places_sources.csv

설정만 바꾸고 바로 실행하세요.
"""

import os
import time
import math
import json
import re
import sys
import random
import requests
import pandas as pd
from typing import List, Dict, Any, Tuple

# =========================
# 🔧 설정
# =========================
REGION_NAME = "도봉구"          # 행정구역 표시용
REGION_ADDR_SUBSTR = "서울 도봉구"  # 주소 필터(카카오)
OUTPUT_MASTER = "places_master.csv"
OUTPUT_SOURCES = "places_sources.csv"

# (선택) Kakao Local API 키: 없으면 빈 문자열로 둠 → OSM만 사용
KAKAO_REST_API_KEY = os.environ.get("KAKAO_REST_API_KEY", "").strip()

# Kakao 사용 여부 자동결정
USE_KAKAO = bool(KAKAO_REST_API_KEY)

# 요청 간 예절 (초)
PAUSE = 0.3

# =========================
# 🧠 카테고리 설계 (세분화 라벨링)
# =========================

# 느좋 (Good Vibe) 세분 카테고리 후보
GV_SUBCATS = [
    # 카페/디저트
    ("독립카페",   ["로스터리", "독립", "핸드드립", "스페셜티"]),
    ("티룸/전통찻집", ["티룸", "전통차", "다도", "tea"]),
    ("뷰카페",     ["루프탑", "뷰", "창가", "전망", "야경"]),
    ("브런치카페", ["브런치", "샌드위치", "베네딕트"]),
    ("포토카페",   ["포토", "사진", "인스타"]),
    # 자연·산책
    ("근린공원",   ["공원", "근린공원", "어린이공원", "소공원"]),
    ("둘레길/숲길",["둘레길", "산책로", "숲길", "트레킹", "하이킹"]),
    ("정원/식물원",["정원", "가든", "식물원", "수목원"]),
    ("하천/강변",  ["하천", "강변", "천", "둔치"]),
    # 전망·조망
    ("전망대",     ["전망대", "전망", "뷰포인트", "전망포인트"]),
    ("야경스팟",   ["야경", "노을", "일몰"]),
    ("루프탑",     ["루프탑", "옥상"]),
    # 문화·학습
    ("작은도서관", ["도서관", "작은도서관"]),
    ("독립서점",   ["서점", "북스", "book"]),
    ("소규모갤러리",["갤러리", "전시장", "아트"]),
    ("공방/체험",  ["공방", "스튜디오", "체험"]),
]

# 숨은 핫플 세분 카테고리 후보
HH_SUBCATS = [
    ("골목카페",     ["골목", "소규모", "작은", "숨은"]),
    ("브런치/디저트", ["브런치", "디저트", "케이크", "타르트"]),
    ("특화카페",     ["수제", "로스터리", "핸드드립", "원두"]),
    ("소규모 한식/분식",["분식", "백반", "칼국수", "국수", "김밥", "덮밥"]),
    ("골목 이탈리안/카레",["파스타", "피자", "리소토", "이탈리안", "카레"]),
    ("가성비 밥집",   ["가성비", "착한", "저렴"]),
    ("작은 갤러리/공방",["갤러리", "공방", "스튜디오"]),
    ("소극장/음악공간",["소극장", "라이브", "음악"]),
]

# 제외/가중치↓ 키워드 (프랜차이즈/소란)
EXCLUDE_KEYWORDS = [
    "스타벅스", "이디야", "더벤티", "메가커피", "빽다방",
    "노래방", "PC방", "멀티방", "유흥", "룸", "대형체인"
]

# =========================
# 🌐 OSM (Overpass) 수집
# =========================
OVERPASS = "https://overpass-api.de/api/interpreter"

OSM_QUERIES = [
    # 카페/도서관/공원/정원/전망대/관광지/산책로 보조
    r"""
    [out:json][timeout:40];
    relation["name"="도봉구"]["admin_level"="6"]["boundary"="administrative"];
    map_to_area->.a;
    (
      node["amenity"="cafe"](area.a);
      node["amenity"="library"](area.a);
      way["leisure"="park"](area.a);
      way["leisure"="garden"](area.a);
      node["tourism"="viewpoint"](area.a);
      node["tourism"="attraction"](area.a);
      way["highway"="footway"](area.a);
      relation["route"="hiking"](area.a);
    );
    out center tags;
    """,
]

def fetch_osm() -> List[Dict[str, Any]]:
    rows = []
    for q in OSM_QUERIES:
        r = requests.post(OVERPASS, data={'data': q})
        r.raise_for_status()
        data = r.json().get("elements", [])
        for e in data:
            tags = e.get("tags", {})
            name = tags.get("name")
            if not name:
                continue
            lat = e.get("lat") or e.get("center", {}).get("lat")
            lon = e.get("lon") or e.get("center", {}).get("lon")
            if not (lat and lon):
                continue
            base_type = tags.get("amenity") or tags.get("leisure") or tags.get("tourism") or tags.get("highway") or tags.get("route") or ""
            rows.append({
                "source": "OSM",
                "name": name,
                "address": "",  # OSM에는 주소가 없을 수 있음
                "lat": float(lat),
                "lon": float(lon),
                "base_type": base_type.upper(),
                "raw": json.dumps(tags, ensure_ascii=False)
            })
        time.sleep(PAUSE)
    return rows

# =========================
# 🟡 Kakao Local (선택) 수집
# =========================

def kakao_search_keyword(query: str, max_pages=5, size=15) -> List[Dict[str, Any]]:
    url = "https://dapi.kakao.com/v2/local/search/keyword.json"
    headers = {"Authorization": f"KakaoAK {KAKAO_REST_API_KEY}"}
    out = []
    for page in range(1, max_pages+1):
        resp = requests.get(url, headers=headers, params={"query": query, "size": size, "page": page}, timeout=10)
        if resp.status_code != 200:
            break
        j = resp.json()
        docs = j.get("documents", [])
        for d in docs:
            addr = d.get("road_address_name") or d.get("address_name") or ""
            if REGION_ADDR_SUBSTR in addr:
                out.append(d)
        if j.get("meta", {}).get("is_end", True):
            break
        time.sleep(PAUSE)
    return out

# 느좋/숨은핫플에 맞춘 “도봉구 전용” 키워드 세트 (확장 가능)
KAKAO_KEYWORDS = [
    # 느좋(무드/자연/전망/문화)
    "도봉구 독립카페", "도봉구 티룸", "도봉구 전통찻집", "도봉구 루프탑 카페",
    "도봉구 브런치 카페", "도봉구 포토카페", "도봉구 조용한 카페",
    "도봉구 공원", "도봉구 소공원", "도봉구 산책로", "도봉구 둘레길", "도봉구 하천",
    "도봉구 전망대", "도봉구 야경",
    "도봉구 작은도서관", "도봉구 독립서점", "도봉구 갤러리", "도봉구 공방",

    # 숨은핫플(골목/소규모 맛집·디저트)
    "도봉구 골목 카페", "도봉구 브런치", "도봉구 디저트", "도봉구 케이크",
    "도봉구 파스타", "도봉구 카레", "도봉구 소극장", "도봉구 스튜디오",
    "도봉구 숨은 맛집", "도봉구 찐맛집"
]

def fetch_kakao() -> List[Dict[str, Any]]:
    if not USE_KAKAO:
        return []
    rows = []
    for kw in KAKAO_KEYWORDS:
        docs = kakao_search_keyword(kw, max_pages=7, size=15)
        for d in docs:
            rows.append({
                "source": f"KAKAO:{kw}",
                "name": d.get("place_name"),
                "address": d.get("road_address_name") or d.get("address_name"),
                "lat": float(d.get("y")),
                "lon": float(d.get("x")),
                "base_type": (d.get("category_group_code") or d.get("category_name") or "").upper(),
                "raw": json.dumps(d, ensure_ascii=False)
            })
        time.sleep(PAUSE)
    return rows

# =========================
# 🧹 정제/중복제거 & 라벨링
# =========================

def is_excluded(name: str) -> bool:
    if not name:
        return True
    for bad in EXCLUDE_KEYWORDS:
        if bad.lower() in name.lower():
            return True
    return False

def dedup(rows: List[Dict[str, Any]]) -> pd.DataFrame:
    df = pd.DataFrame(rows)
    if df.empty:
        return df
    # 좌표 기반 라운딩으로 근접 중복 병합
    df["lat_r"] = df["lat"].round(5)
    df["lon_r"] = df["lon"].round(5)
    # 이름/좌표/주소 조합으로 중복 제거
    subset = ["name", "lat_r", "lon_r"]
    if "address" in df.columns:
        subset = ["name", "address", "lat_r", "lon_r"]
    df = df.drop_duplicates(subset=subset)
    # 제외 키워드 제거
    df = df[~df["name"].apply(is_excluded)].copy()
    # 주소가 비어있고 카카오에서 동일 좌표가 있으면 주소 채우기 (간단 병합)
    return df.drop(columns=["lat_r","lon_r"])

def tag_subcategories(name: str, base_type: str) -> Tuple[str, str]:
    """세분 카테고리 (느좋/숨은핫플) 라벨링; 가장 먼저 매칭되는 하나씩 부여"""
    # 느좋 라벨
    for gv_cat, kws in GV_SUBCATS:
        if any(kw.lower() in name.lower() for kw in kws):
            return ("느좋", gv_cat)
    # 숨은핫플 라벨
    for hh_cat, kws in HH_SUBCATS:
        if any(kw.lower() in name.lower() for kw in kws):
            return ("숨은핫플", hh_cat)
    # 베이스 타입으로 힌트
    bt = (base_type or "").lower()
    if "cafe" in bt or "ce7" in bt:
        return ("느좋", "독립카페")
    if "library" in bt:
        return ("느좋", "작은도서관")
    if "park" in bt or "garden" in bt:
        return ("느좋", "근린공원")
    if "viewpoint" in bt or "attraction" in bt:
        return ("느좋", "전망대")
    if "restaurant" in bt or "fd6" in bt:
        return ("숨은핫플", "가성비 밥집")
    if "footway" in bt or "hiking" in bt:
        return ("느좋", "둘레길/숲길")
    return ("미분류", "미분류")

def main():
    print(f"[i] 도봉구 장소 추출 시작 (Kakao 사용: {USE_KAKAO})")
    all_rows = []

    # 1) OSM
    try:
        osm_rows = fetch_osm()
        print(f"[OSM] 수집: {len(osm_rows)}")
        all_rows.extend(osm_rows)
    except Exception as e:
        print("[OSM] 오류:", e)

    # 2) Kakao (선택)
    if USE_KAKAO:
        try:
            kakao_rows = fetch_kakao()
            print(f"[KAKAO] 수집: {len(kakao_rows)}")
            all_rows.extend(kakao_rows)
        except Exception as e:
            print("[KAKAO] 오류:", e)
            print("  ↳ Kakao 키를 비우고 OSM만으로도 실행 가능합니다.")

    if not all_rows:
        print("수집 결과가 없습니다. 네트워크/방화벽/키 설정을 확인하세요.")
        sys.exit(1)

    # 3) 정제/중복 제거
    df = dedup(all_rows)
    if df.empty:
        print("정제 후 결과가 없습니다.")
        sys.exit(1)

    # 4) 라벨링
    labels = df.apply(lambda r: tag_subcategories(str(r.get("name","")), str(r.get("base_type",""))), axis=1)
    df["top_category"] = [t[0] for t in labels]      # 느좋/숨은핫플/미분류
    df["sub_category"] = [t[1] for t in labels]

    # 5) 주소 없으면 공백 문자열 유지
    if "address" not in df.columns:
        df["address"] = ""

    # 6) 정렬·열 순서
    cols = ["name","address","lat","lon","top_category","sub_category","base_type","source","raw"]
    for c in cols:
        if c not in df.columns:
            df[c] = ""
    df = df[cols]

    # 7) 원천 로그 테이블도 생성
    df_sources = pd.DataFrame(all_rows)
    if not df_sources.empty:
        df_sources.to_csv(OUTPUT_SOURCES, index=False, encoding="utf-8-sig")

    # 8) 저장
    #df.to_csv(OUTPUT_MASTER, index=False, encoding="utf-8-sig")
    #print(f"[✔] 완료: {OUTPUT_MASTER} (행 {len(df)})")
    #if not df_sources.empty:
    #    print(f"[i] 원천 로그: {OUTPUT_SOURCES} (행 {len(df_sources)})")

if __name__ == "__main__":
    main()


[i] 도봉구 장소 추출 시작 (Kakao 사용: False)
[OSM] 수집: 112


In [7]:
# 2_naver_crawl_small.py
import time, random, pandas as pd, re
from urllib.parse import quote_plus
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchFrameException, NoSuchElementException, WebDriverException

SEED_CSV = "dobong_seed.csv"
MAX_PLACES = 15         # 장소 최대 15곳만(학습용)
LINKS_PER_PLACE = 5     # 링크 상한
SLEEP = lambda: time.sleep(random.uniform(1.5, 3.5))

# 크롬 드라이버 준비
options = webdriver.ChromeOptions()
options.add_argument("--headless=new")
options.add_argument("--window-size=1280,1800")
driver = webdriver.Chrome(options=options)

def naver_search_links(place_name):
    # 도봉구 + 품질 키워드로 검색
    q = f"{place_name} 도봉구 후기 분위기 조용 뷰 힐링 추천"
    url = f"https://search.naver.com/search.naver?sm=tab_hty.top&where=view&query={quote_plus(q)}"
    driver.get(url); SLEEP()
    links=[]
    # 뷰탭에서 블로그/포스트 카드의 a 태그 수집
    cards = driver.find_elements(By.CSS_SELECTOR, "a.api_txt_lines.total_tit")
    for a in cards[:LINKS_PER_PLACE*2]:  # 예비 링크 넉넉히
        href = a.get_attribute("href")
        if not href:
            continue
        if "blog.naver.com" in href or "m.blog.naver.com" in href:
            links.append(href)
        if len(links) >= LINKS_PER_PLACE:
            break
    return links

def extract_naver_blog_text(url):
    # 블로그 본문은 iframe(mainFrame) 내부
    driver.get(url); SLEEP()
    txt = ""
    try:
        # 구형 스킨
        driver.switch_to.frame("mainFrame")
        SLEEP()
        html = driver.page_source
        soup = BeautifulSoup(html, "lxml")
        # 여러 스킨 대응
        cand = [
            "#postViewArea", "#post-view", ".se-main-container", ".se-viewer", "#content-area"
        ]
        for sel in cand:
            el = soup.select_one(sel)
            if el:
                txt = el.get_text(" ", strip=True)
                break
        driver.switch_to.default_content()
    except NoSuchFrameException:
        # 최신 에디터(m.blog) 등은 본문이 바로 존재할 수 있음
        html = driver.page_source
        soup = BeautifulSoup(html, "lxml")
        el = soup.select_one(".se-main-container") or soup.select_one("#post-view") or soup.select_one("#content")
        if el:
            txt = el.get_text(" ", strip=True)
    except WebDriverException:
        pass
    # 광고/이모지/해시태그 일부 정리
    txt = re.sub(r"#[^\s]+", " ", txt)
    txt = re.sub(r"\s+", " ", txt).strip()
    return txt

seed = pd.read_csv(SEED_CSV).head(MAX_PLACES)
rows=[]
for _, r in seed.iterrows():
    name = r["name"]
    links = naver_search_links(name)
    for lk in links:
        body = extract_naver_blog_text(lk)
        if len(body) < 100:  # 너무 짧으면 제외
            continue
        rows.append({"place": name, "url": lk, "text": body})
        print(f"[ok] {name} ← {lk[:60]}... ({len(body)} chars)")
        SLEEP()

driver.quit()
pd.DataFrame(rows).to_csv("dobong_texts.csv", index=False, encoding="utf-8-sig")
print("수집 텍스트 수:", len(rows))


수집 텍스트 수: 0


In [15]:
import time, random, pandas as pd, re
from urllib.parse import quote_plus
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchFrameException, NoSuchElementException, WebDriverException

SEED_CSV = "dobong_seed.csv"
MAX_PLACES = 15        # 장소 최대 15곳만(테스트용)
LINKS_PER_PLACE = 5    # 장소당 수집할 최대 링크 수
SLEEP = lambda: time.sleep(random.uniform(1.5, 3.5))

# 크롬 드라이버 준비
options = webdriver.ChromeOptions()
# options.add_argument("--headless=new")  # 필요 시 주석 해제하여 백그라운드에서 실행
options.add_argument("--window-size=1280,1800")
driver = webdriver.Chrome(options=options)

def naver_search_links(place_name):
    """네이버 VIEW 탭에서 특정 장소의 블로그 리뷰 링크를 수집합니다."""
    # 도봉구 + 품질 키워드로 검색하여 더 정확한 결과 유도
    q = f"{place_name} 도봉구 추천"
    url = f"https://search.naver.com/search.naver?sm=tab_hty.top&where=view&query={quote_plus(q)}"
    
    driver.get(url)
    SLEEP()
    
    links = []
    # 뷰탭에서 블로그/포스트 카드의 a 태그 수집
    # [수정됨] 네이버의 HTML 구조 변경에 따라 선택자를 'a.title_link'로 변경
    cards = driver.find_elements(By.CSS_SELECTOR, "a.title_link")
    
    for a in cards[:LINKS_PER_PLACE * 2]: # 예비 링크 넉넉히 수집
        href = a.get_attribute("href")
        if not href:
            continue
        # 네이버 블로그 링크만 필터링
        if "blog.naver.com" in href or "m.blog.naver.com" in href:
            links.append(href)
        # 목표한 링크 개수를 채우면 중단
        if len(links) >= LINKS_PER_PLACE:
            break
            
    return links

def extract_naver_blog_text(url):
    """네이버 블로그 URL에 접속하여 본문 텍스트를 추출합니다."""
    driver.get(url)
    SLEEP()
    
    txt = ""
    try:
        # 구형 에디터는 본문이 iframe 내부에 있음
        driver.switch_to.frame("mainFrame")
        html = driver.page_source
        soup = BeautifulSoup(html, "lxml")
        
        # 여러 종류의 블로그 스킨에 대응하기 위한 선택자 목록
        content_selectors = [
            "#postViewArea",          # 구형 에디터
            ".se-main-container",     # 스마트 에디터 ONE (PC)
            ".se_component_wrap",     # 구형 스마트 에디터
            "#post-view"              # 다른 스킨
        ]
        
        for sel in content_selectors:
            el = soup.select_one(sel)
            if el:
                txt = el.get_text(" ", strip=True)
                break
        
        driver.switch_to.default_content()

    except NoSuchFrameException:
        # 최신 에디터(m.blog 등)는 iframe 없이 본문이 바로 존재
        html = driver.page_source
        soup = BeautifulSoup(html, "lxml")
        el = soup.select_one(".se-main-container") or soup.select_one("#post-view")
        if el:
            txt = el.get_text(" ", strip=True)
            
    except WebDriverException as e:
        print(f"   [!] WebDriver 오류 발생: {url} ({e})")
        pass
        
    # 불필요한 공백, 해시태그 등 정리
    if txt:
        txt = re.sub(r"#[^\s]+", " ", txt)
        txt = re.sub(r"\s+", " ", txt).strip()
        
    return txt

# --- 메인 실행 로직 ---
try:
    seed = pd.read_csv(SEED_CSV).head(MAX_PLACES)
    rows = []
    
    print(f"총 {len(seed)}개 장소에 대한 블로그 텍스트 수집을 시작합니다.")

    for i, r in seed.iterrows():
        name = r["name"]
        print(f"\n({i+1}/{len(seed)}) '{name}' 링크 수집 중...")
        
        links = naver_search_links(name)
        if not links:
            print(f"   [!] '{name}'에 대한 유효한 블로그 링크를 찾지 못했습니다.")
            continue
            
        print(f"   - 찾은 링크 {len(links)}개. 본문 추출 시작...")
        
        for lk in links:
            body = extract_naver_blog_text(lk)
            if len(body) < 100: # 너무 짧은 글은 광고 등이므로 제외
                print(f"   - (skip) 너무 짧은 글: {lk[:60]}...")
                continue
            
            rows.append({"place": name, "url": lk, "text": body})
            print(f"   - [ok] {name} ← {lk[:60]}... ({len(body)} chars)")
            SLEEP()

    # 최종 결과 저장
    if rows:
        pd.DataFrame(rows).to_csv("dobong_texts.csv", index=False, encoding="utf-8-sig")
        print(f"\n[✔] 수집 완료! 총 {len(rows)}개의 텍스트를 'dobong_texts.csv'에 저장했습니다.")
    else:
        print("\n[!] 최종적으로 수집된 텍스트가 없습니다. 검색어나 선택자를 확인하세요.")

except FileNotFoundError:
    print(f"[오류] '{SEED_CSV}' 파일을 찾을 수 없습니다. 파일이 올바른 위치에 있는지 확인하세요.")
    
finally:
    # 모든 작업이 끝나면 드라이버 종료
    driver.quit()

총 15개 장소에 대한 블로그 텍스트 수집을 시작합니다.

(1/15) '창4동새마을문고' 링크 수집 중...
   [!] '창4동새마을문고'에 대한 유효한 블로그 링크를 찾지 못했습니다.

(2/15) '쌍문3동문고' 링크 수집 중...
   [!] '쌍문3동문고'에 대한 유효한 블로그 링크를 찾지 못했습니다.

(3/15) '창2동새마을문고' 링크 수집 중...
   [!] '창2동새마을문고'에 대한 유효한 블로그 링크를 찾지 못했습니다.

(4/15) '창3동새마을문고' 링크 수집 중...
   [!] '창3동새마을문고'에 대한 유효한 블로그 링크를 찾지 못했습니다.

(5/15) '도봉2동문고' 링크 수집 중...
   [!] '도봉2동문고'에 대한 유효한 블로그 링크를 찾지 못했습니다.

(6/15) '방학1동문고' 링크 수집 중...
   [!] '방학1동문고'에 대한 유효한 블로그 링크를 찾지 못했습니다.

(7/15) '방학2동문고' 링크 수집 중...
   [!] '방학2동문고'에 대한 유효한 블로그 링크를 찾지 못했습니다.

(8/15) '방학3동문고' 링크 수집 중...
   [!] '방학3동문고'에 대한 유효한 블로그 링크를 찾지 못했습니다.

(9/15) '방학4동문고' 링크 수집 중...
   [!] '방학4동문고'에 대한 유효한 블로그 링크를 찾지 못했습니다.

(10/15) '쌍문1동문고' 링크 수집 중...
   [!] '쌍문1동문고'에 대한 유효한 블로그 링크를 찾지 못했습니다.

(11/15) '창1동문고' 링크 수집 중...
   [!] '창1동문고'에 대한 유효한 블로그 링크를 찾지 못했습니다.

(12/15) '쌍문4동문고' 링크 수집 중...
   [!] '쌍문4동문고'에 대한 유효한 블로그 링크를 찾지 못했습니다.

(13/15) '서울특별시립도봉도서관' 링크 수집 중...
   [!] '서울특별시립도봉도서관'에 대한 유효한 블로그 링크를 찾지 못했습니다.

(14/15) '덕성여자대학교 도서관' 링크 수집 중...


In [11]:
# dobong_places_simple.py
import time, json, re, requests, pandas as pd

OVERPASS = "https://overpass.kumi.systems/api/interpreter"

# 도봉구 경계 내에서 카페/베이커리/도서관/공원/정원/전망대/산책로/둘레길 포괄 수집
QUERY = r"""
[out:json][timeout:45];
relation["name"="도봉구"]["admin_level"="6"]["boundary"="administrative"];
map_to_area->.a;
(
  node["amenity"="cafe"](area.a);
  node["shop"="bakery"](area.a);
  node["amenity"="library"](area.a);
  way["leisure"="park"](area.a);
  way["leisure"="garden"](area.a);
  node["tourism"="viewpoint"](area.a);
  node["tourism"="attraction"](area.a);
  way["highway"="footway"]["name"](area.a);        /* 이름 있는 산책로 */
  relation["route"="hiking"](area.a);              /* 둘레길/트레킹 */
);
out center tags;
"""

EXCLUDE_WORDS = [
    "스타벅스","이디야","빽다방","메가커피","더벤티",  # 프랜차이즈
    "노래방","PC방","멀티방","유흥","룸"             # 소음/유흥
]

# 간단 라벨링 규칙(상위/세부)
def label_category(tags: dict, name: str):
    amenity = tags.get("amenity","")
    leisure = tags.get("leisure","")
    tourism = tags.get("tourism","")
    shop    = tags.get("shop","")
    highway = tags.get("highway","")
    route   = tags.get("route","")
    nlow = (name or "").lower()

    # 기본 타입 판정
    if amenity == "cafe" or shop == "bakery":
        base = "카페/베이커리"
    elif amenity == "library":
        base = "도서관"
    elif leisure in ("park","garden"):
        base = "공원/정원"
    elif tourism in ("viewpoint","attraction"):
        base = "전망/명소"
    elif highway == "footway" or route == "hiking":
        base = "산책로/둘레길"
    else:
        base = "기타"

    # 상위 카테고리(느좋/숨은핫플) & 세부
    top, sub = "미분류","미분류"
    # 느좋 신호
    if base in ("공원/정원","산책로/둘레길","도서관","전망/명소"):
        top = "느좋"
        if base == "공원/정원": sub = "근린공원/정원"
        elif base == "산책로/둘레길": sub = "둘레길/숲길"
        elif base == "도서관": sub = "작은도서관"
        elif base == "전망/명소":
            if any(k in nlow for k in ["야경","노을","일몰"]): sub = "야경스팟"
            else: sub = "전망대"
    # 카페/베이커리는 느좋/숨은핫플 둘 다 가능 → 이름 키워드로 구분 시도
    if base == "카페/베이커리":
        if any(k in nlow for k in ["티룸","전통","다도","tea","루프탑","뷰","창가","사진","포토","감성","브런치","로스터리","핸드드립","스페셜티"]):
            top, sub = "느좋","독립/감성카페"
        # 숨은핫플 후보 키워드(골목·소규모·디저트 특화 등)
        if any(k in nlow for k in ["골목","소규모","작은","디저트","케이크","타르트","브런치"]):
            top, sub = "숨은핫플","골목/디저트카페"

    # 맛집 소수 포착(여기선 OSM 한정이라 빈약하지만 최소 규칙)
    if tags.get("amenity") == "restaurant":
        top, sub = "숨은핫플","소규모 식당"

    return base, top, sub

def fetch_overpass(q: str):
    r = requests.post(OVERPASS, data={'data': q}, timeout=60)
    r.raise_for_status()
    return r.json().get("elements", [])

def main():
    print("[i] 도봉구 장소 수집 중…")
    elems = fetch_overpass(QUERY)
    rows = []
    for e in elems:
        tags = e.get("tags", {})
        name = tags.get("name")
        if not name:
            continue
        if any(bad in name for bad in EXCLUDE_WORDS):
            continue
        lat = e.get("lat") or (e.get("center") or {}).get("lat")
        lon = e.get("lon") or (e.get("center") or {}).get("lon")
        if not (lat and lon):
            continue

        base, top, sub = label_category(tags, name)
        rows.append({
            "name": name,
            "lat": float(lat),
            "lon": float(lon),
            "base_type": base,
            "top_category": top,      # 느좋 / 숨은핫플 / 미분류
            "sub_category": sub,      # 세부 라벨
            "raw_tags": json.dumps(tags, ensure_ascii=False)
        })

    if not rows:
        print("수집 결과가 없습니다. 네트워크/Overpass 상태를 확인하세요.")
        return

    df = pd.DataFrame(rows)

    # 근접 중복 제거(이름+좌표 라운딩)
    df["lat_r"] = df["lat"].round(5)
    df["lon_r"] = df["lon"].round(5)
    df = df.drop_duplicates(subset=["name","lat_r","lon_r"]).drop(columns=["lat_r","lon_r"])

    # 정렬
    order = ["name","lat","lon","base_type","top_category","sub_category","raw_tags"]
    df = df[order].sort_values(["top_category","sub_category","name"]).reset_index(drop=True)

    df.to_csv("dobong_places.csv", index=False, encoding="utf-8-sig")
    print(f"[✔] 완료: dobong_places.csv (행 {len(df)})")

if __name__ == "__main__":
    main()
print(df.head(10).to_string(index=False))    # 앞 10행 미리보기
print(df["top_category"].value_counts())     # 카테고리별 개수
print(df)


[i] 도봉구 장소 수집 중…
[✔] 완료: dobong_places.csv (행 118)
    name  address       lat        lon top_category sub_category base_type source                                                                                                                                                                                                                        raw
창4동새마을문고      NaN 37.652259 127.051481           느좋        작은도서관   LIBRARY    OSM     {"amenity": "library", "name": "창4동새마을문고", "name:en": "Chang 4 Dongsae Village Library", "name:ko": "창4동새마을문고", "name:ko-Latn": "Chang 4 Dongsaemaeulmungo", "ncat": "도서관", "source": "http://kr.open.gugi.yahoo.com"}
  쌍문3동문고      NaN 37.648910 127.028004           느좋        작은도서관   LIBRARY    OSM                                                                                {"amenity": "library", "name": "쌍문3동문고", "name:en": "Ssangmun 3 Dong Library", "name:ko": "쌍문3동문고", "name:ko-Latn": "Ssangmun 3 Dongmungo"}
창2동새마을문고      NaN 37.641379 127.035482        

In [9]:
import pandas as pd

df = pd.read_csv("places_master.csv")
print(df.head(10))   # 앞 10개 행 미리보기

# 카테고리별 개수 확인
print(df["top_category"].value_counts())


       name  address        lat         lon top_category sub_category  \
0  창4동새마을문고      NaN  37.652259  127.051481           느좋        작은도서관   
1    쌍문3동문고      NaN  37.648910  127.028004           느좋        작은도서관   
2  창2동새마을문고      NaN  37.641379  127.035482           느좋        작은도서관   
3  창3동새마을문고      NaN  37.637273  127.042516           느좋        작은도서관   
4    도봉2동문고      NaN  37.669710  127.046498           느좋        작은도서관   
5    방학1동문고      NaN  37.664246  127.040617           느좋        작은도서관   
6    방학2동문고      NaN  37.668283  127.034999           느좋        작은도서관   
7    방학3동문고      NaN  37.659213  127.027873           느좋        작은도서관   
8    방학4동문고      NaN  37.659888  127.023421           느좋        작은도서관   
9    쌍문1동문고      NaN  37.648006  127.025980           느좋        작은도서관   

  base_type source                                                raw  
0   LIBRARY    OSM  {"amenity": "library", "name": "창4동새마을문고", "na...  
1   LIBRARY    OSM  {"amenity": "library", "name": "

In [10]:
import time, json, re, requests, pandas as pd

OVERPASS = "https://overpass-api.de/api/interpreter"

# 도봉구 경계 내에서 카페/베이커리/도서관/공원/정원/전망대/산책로/둘레길 + 핫플 후보군 포괄 수집
QUERY = r"""
[out:json][timeout:60];
relation["name"="도봉구"]["admin_level"="6"]["boundary"="administrative"];
map_to_area->.a;
(
  /* --- 기존 "느좋" 카테고리 --- */
  node["amenity"="cafe"](area.a);
  way["amenity"="cafe"](area.a);
  node["shop"="bakery"](area.a);
  way["shop"="bakery"](area.a);
  node["amenity"="library"](area.a);
  way["leisure"="park"](area.a);
  way["leisure"="garden"](area.a);
  node["tourism"="viewpoint"](area.a);
  node["tourism"="attraction"](area.a);
  way["highway"="footway"]["name"](area.a);      /* 이름 있는 산책로 */
  relation["route"="hiking"](area.a);           /* 둘레길/트레킹 */

  /* --- "핫플" 후보군 추가 --- */
  node["amenity"="restaurant"](area.a);
  way["amenity"="restaurant"](area.a);
  node["amenity"~"bar|pub"](area.a);
  way["amenity"~"bar|pub"](area.a);
  node["shop"="books"](area.a);
  way["shop"="books"](area.a);
  node["tourism"="gallery"](area.a);
  way["tourism"="gallery"](area.a);
  node["amenity"="arts_centre"](area.a);
  way["amenity"="arts_centre"](area.a);
);
out center tags;
"""

EXCLUDE_WORDS = [
    "스타벅스","이디야","빽다방","메가커피","더벤티",   # 프랜차이즈
    "노래방","PC방","멀티방","유흥","룸", "단란주점"   # 소음/유흥
]

# 확장된 라벨링 규칙(상위/세부)
def label_category(tags: dict, name: str):
    amenity = tags.get("amenity","")
    leisure = tags.get("leisure","")
    tourism = tags.get("tourism","")
    shop    = tags.get("shop","")
    highway = tags.get("highway","")
    route   = tags.get("route","")
    nlow = (name or "").lower()

    # 기본 타입 판정
    if amenity == "cafe" or shop == "bakery":
        base = "카페/베이커리"
    elif amenity == "library":
        base = "도서관"
    elif leisure in ("park","garden"):
        base = "공원/정원"
    elif tourism in ("viewpoint","attraction"):
        base = "전망/명소"
    elif highway == "footway" or route == "hiking":
        base = "산책로/둘레길"
    elif amenity == "restaurant":
        base = "식당"
    elif amenity in ("bar", "pub"):
        base = "바/펍"
    elif shop == "books":
        base = "서점"
    elif tourism == "gallery" or amenity == "arts_centre":
        base = "문화공간"
    else:
        base = "기타"

    # 상위 카테고리(느좋/숨은핫플) & 세부
    top, sub = "미분류","미분류"

    # --- 1. "느좋" 카테고리 분류 ---
    if base in ("공원/정원","산책로/둘레길","도서관","전망/명소"):
        top = "느좋"
        if base == "공원/정원": sub = "근린공원/정원"
        elif base == "산책로/둘레길": sub = "둘레길/숲길"
        elif base == "도서관": sub = "작은도서관"
        elif base == "전망/명소":
            if any(k in nlow for k in ["야경","노을","일몰"]): sub = "야경스팟"
            else: sub = "전망대"

    # --- 2. "카페/베이커리"는 키워드로 "느좋"과 "숨은핫플" 구분 ---
    if base == "카페/베이커리":
        # '느좋' 신호 키워드
        if any(k in nlow for k in ["티룸","전통","다도","tea","루프탑","뷰","창가","사진","포토","감성","브런치","로스터리","핸드드립","스페셜티"]):
            top, sub = "느좋","독립/감성카페"
        # '숨은핫플' 신호 키워드
        elif any(k in nlow for k in ["골목","소규모","작은","디저트","케이크","타르트", "베이킹", "클래스", "공방"]):
            top, sub = "숨은핫플","골목/디저트카페"

    # --- 3. "숨은핫플" 카테고리 분류 (신규 추가된 장소 유형) ---
    if base == "식당":
        top, sub = "숨은핫플", "개성있는 식당"
    elif base == "바/펍":
        top, sub = "숨은핫플", "특색있는 바/펍"
    elif base == "서점":
        # 이름에 '독립서점' 등의 키워드가 있으면 더 확실하지만, OSM 데이터상 보통 일반 서점과 함께 분류되므로 우선 모두 포함
        top, sub = "숨은핫플", "독립/테마서점"
    elif base == "문화공간":
        top, sub = "숨은핫플", "갤러리/문화공간"

    return base, top, sub

def fetch_overpass(q: str):
    print("[i] Overpass API에 데이터 요청 중...")
    try:
        r = requests.post(OVERPASS, data={'data': q}, timeout=90)
        r.raise_for_status()
        return r.json().get("elements", [])
    except requests.exceptions.RequestException as e:
        print(f"[!] API 요청 실패: {e}")
        return []

def main():
    print("[i] 도봉구 장소 수집 시작…")
    elems = fetch_overpass(QUERY)
    if not elems:
        print("[!] 수집된 데이터가 없습니다. 쿼리나 네트워크 상태를 확인하세요.")
        return

    rows = []
    for e in elems:
        tags = e.get("tags", {})
        name = tags.get("name")
        if not name or any(bad in name for bad in EXCLUDE_WORDS):
            continue

        lat = e.get("lat") or (e.get("center") or {}).get("lat")
        lon = e.get("lon") or (e.get("center") or {}).get("lon")
        if not (lat and lon):
            continue

        base, top, sub = label_category(tags, name)
        rows.append({
            "name": name,
            "lat": float(lat),
            "lon": float(lon),
            "base_type": base,
            "top_category": top,      # 느좋 / 숨은핫플 / 미분류
            "sub_category": sub,      # 세부 라벨
            "raw_tags": json.dumps(tags, ensure_ascii=False)
        })

    if not rows:
        print("수집 및 필터링 결과가 없습니다.")
        return

    df = pd.DataFrame(rows)

    # 근접 중복 제거 (이름 + 좌표 소수점 5자리 기준)
    df["lat_r"] = df["lat"].round(5)
    df["lon_r"] = df["lon"].round(5)
    df = df.drop_duplicates(subset=["name","lat_r","lon_r"]).drop(columns=["lat_r","lon_r"])

    # 정렬
    order = ["name","lat","lon","base_type","top_category","sub_category","raw_tags"]
    df = df[order].sort_values(["top_category","sub_category","name"]).reset_index(drop=True)

    # "미분류" 데이터는 제외하고 최종 결과물 생성
    df_filtered = df[df["top_category"] != "미분류"].reset_index(drop=True)

    if df_filtered.empty:
        print("[!] 분류된 장소가 없습니다.")
        return

    output_filename = "dobong_places_classified.csv"
    df_filtered.to_csv(output_filename, index=False, encoding="utf-8-sig")
    print(f"\n[✔] 완료: {output_filename} (총 {len(df_filtered)}곳)")

    # --- 결과 미리보기 ---
    print("\n--- 상위 10개 데이터 미리보기 ---")
    print(df_filtered.head(10).to_string(index=False))

    print("\n--- 카테고리별 장소 개수 ---")
    print(df_filtered["top_category"].value_counts())

    print("\n--- 세부 카테고리별 장소 개수 ---")
    print(df_filtered.groupby(["top_category", "sub_category"]).size().unstack(fill_value=0))


if __name__ == "__main__":
    main()

[i] 도봉구 장소 수집 시작…
[i] Overpass API에 데이터 요청 중...

[✔] 완료: dobong_places_classified.csv (총 146곳)

--- 상위 10개 데이터 미리보기 ---
     name       lat        lon base_type top_category sub_category                                                                                                                                   raw_tags
I'PARK 공원 37.660194 127.045378     공원/정원           느좋      근린공원/정원                                                               {"leisure": "park", "name": "I'PARK 공원", "name:en": "I'PARK Apartment Park"}
갈대밭 어린이공원 37.674346 127.045815     공원/정원           느좋      근린공원/정원                                                                                                   {"leisure": "park", "name": "갈대밭 어린이공원"}
 개나리어린이공원 37.667345 127.037125     공원/정원           느좋      근린공원/정원                                                                                                    {"leisure": "park", "name": "개나리어린이공원"}
 금성윗들 소공원 37.667665 127.044776     공원/정원           느좋   

In [16]:
# 1_extract_praise_dna.py
import time, random, pandas as pd, re
from urllib.parse import quote_plus
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchFrameException, NoSuchElementException, WebDriverException

# --- 설정 ---
# 1단계에서 직접 만든 '대표 선수' 목록 파일
SEED_CSV = "reference_places.csv"
# '칭찬 DNA' 분석의 원재료가 될 결과 파일
OUTPUT_CSV = "reference_texts.csv"

MAX_PLACES = 30        # 대표 장소는 넉넉하게 30곳까지
LINKS_PER_PLACE = 5    # 장소당 수집할 최대 링크 수
SLEEP = lambda: time.sleep(random.uniform(1.5, 3.5))

# --- 크롬 드라이버 설정 ---
options = webdriver.ChromeOptions()
# options.add_argument("--headless=new")  # 필요 시 주석 해제하여 백그라운드에서 실행
options.add_argument("--window-size=1280,1800")
driver = webdriver.Chrome(options=options)

def naver_search_links(place_name):
    """네이버 VIEW 탭에서 특정 장소의 블로그 리뷰 링크를 수집합니다."""
    # 대표 장소 검색 시에는 지역명(도봉구)을 빼서 범용적으로 검색
    q = f"{place_name} 후기 분위기 뷰 추천"
    url = f"https://search.naver.com/search.naver?sm=tab_hty.top&where=view&query={quote_plus(q)}"
    
    driver.get(url)
    SLEEP()
    
    links = []
    # 네이버 VIEW탭 제목 링크 선택자
    cards = driver.find_elements(By.CSS_SELECTOR, "a.title_link")
    
    for a in cards[:LINKS_PER_PLACE * 2]:
        href = a.get_attribute("href")
        if not href:
            continue
        if "blog.naver.com" in href or "m.blog.naver.com" in href:
            links.append(href)
        if len(links) >= LINKS_PER_PLACE:
            break
            
    return links

def extract_naver_blog_text(url):
    """네이버 블로그 URL에 접속하여 본문 텍스트를 추출합니다."""
    driver.get(url)
    SLEEP()
    
    txt = ""
    try:
        driver.switch_to.frame("mainFrame")
        html = driver.page_source
        soup = BeautifulSoup(html, "lxml")
        
        content_selectors = ["#postViewArea", ".se-main-container", ".se_component_wrap", "#post-view"]
        for sel in content_selectors:
            el = soup.select_one(sel)
            if el:
                txt = el.get_text(" ", strip=True)
                break
        driver.switch_to.default_content()

    except NoSuchFrameException:
        html = driver.page_source
        soup = BeautifulSoup(html, "lxml")
        el = soup.select_one(".se-main-container") or soup.select_one("#post-view")
        if el:
            txt = el.get_text(" ", strip=True)
            
    except WebDriverException as e:
        print(f"   [!] WebDriver 오류 발생: {url} ({e})")
        pass
        
    if txt:
        txt = re.sub(r"#[^\s]+", " ", txt)
        txt = re.sub(r"\s+", " ", txt).strip()
        
    return txt

# --- 메인 실행 로직 ---
try:
    seed = pd.read_csv(SEED_CSV).head(MAX_PLACES)
    rows = []
    
    print(f"--- '칭찬 DNA' 추출을 위한 텍스트 수집 시작 ---")
    print(f"'{SEED_CSV}' 파일에서 총 {len(seed)}개 대표 장소를 읽어왔습니다.")

    for i, r in seed.iterrows():
        name = r["name"]
        print(f"\n({i+1}/{len(seed)}) '{name}' 링크 수집 중...")
        
        links = naver_search_links(name)
        if not links:
            print(f"   [!] '{name}'에 대한 유효한 블로그 링크를 찾지 못했습니다.")
            continue
            
        print(f"   - 찾은 링크 {len(links)}개. 본문 추출 시작...")
        
        for lk in links:
            body = extract_naver_blog_text(lk)
            if len(body) < 100:
                print(f"   - (skip) 너무 짧은 글: {lk[:60]}...")
                continue
            
            rows.append({"place": name, "url": lk, "text": body})
            print(f"   - [ok] {name} ← {lk[:60]}... ({len(body)} chars)")
            SLEEP()

    if rows:
        pd.DataFrame(rows).to_csv(OUTPUT_CSV, index=False, encoding="utf-8-sig")
        print(f"\n[✔] 수집 완료! 총 {len(rows)}개의 텍스트를 '{OUTPUT_CSV}'에 저장했습니다.")
    else:
        print(f"\n[!] 최종적으로 수집된 텍스트가 없습니다. '{SEED_CSV}' 파일 내용과 검색어를 확인하세요.")

except FileNotFoundError:
    print(f"[오류] '{SEED_CSV}' 파일을 찾을 수 없습니다. 아래 내용을 참고하여 파일을 생성해주세요.")
    
finally:
    driver.quit()

[오류] 'reference_places.csv' 파일을 찾을 수 없습니다. 아래 내용을 참고하여 파일을 생성해주세요.


############핫플 장소 찾기########



In [1]:
# 1_discover_representatives_v3.py
import time, random, pandas as pd
from urllib.parse import quote_plus
from selenium import webdriver
from selenium.webdriver.common.by import By

# --- 설정 ---
DISCOVERY_QUERIES = [
    "핫플", "감성 장소", "맛집",
    "가볼만한 곳", "인스타 감성 장소", "추천 장소", "데이트" ,"느좋 장소"
]
OUTPUT_CSV = "discovered_places.csv"
MAX_REPRESENTATIVES = 20
SLEEP = lambda: time.sleep(random.uniform(1.5, 3.0))

# --- 크롬 드라이버 설정 ---
options = webdriver.ChromeOptions()
# options.add_argument("--headless=new")
options.add_argument("--window-size=1280,1800")
driver = webdriver.Chrome(options=options)

def discover_representative_places(queries):
    place_names = set()
    print("--- 1단계: 대표 선수 장소 발굴 시작 (v3) ---")
    
    for q in queries:
        if len(place_names) >= MAX_REPRESENTATIVES: break
        print(f"\n[탐색] 검색어: '{q}'")
        url = f"https://search.naver.com/search.naver?sm=tab_hty.top&where=view&query={quote_plus(q)}"
        driver.get(url)
        SLEEP()
        
        # [수정됨] 네이버 구조 변경에 따라 선택자를 'a.place_name' -> 'a.sub_name'으로 변경
        place_links = driver.find_elements(By.CSS_SELECTOR, "a.sub_name")
        
        if not place_links:
            print("   [!] 해당 검색어에서 장소 이름을 직접 추출하지 못했습니다.")
            continue
            
        for link in place_links:
            try:
                name = link.text
                if name and 3 < len(name) < 20:
                    print(f"   [발견] '{name}'")
                    place_names.add(name)
                    if len(place_names) >= MAX_REPRESENTATIVES: break
            except Exception:
                continue

    print(f"\n[결과] 총 {len(place_names)}개의 대표 선수를 발굴했습니다.")
    return list(place_names)

try:
    representative_places = discover_representative_places(DISCOVERY_QUERIES)
    
    if representative_places:
        df = pd.DataFrame(representative_places, columns=["name"])
        df.to_csv(OUTPUT_CSV, index=False, encoding="utf-8-sig")
        print(f"\n[✔] 완료! '{OUTPUT_CSV}' 파일에 대표 선수 목록을 저장했습니다.")
    else:
        print("\n[!] 대표 선수를 찾지 못했습니다. 검색어를 변경하거나, 다른 전략을 시도해보세요.")
        
finally:
    driver.quit()

--- 1단계: 대표 선수 장소 발굴 시작 (v3) ---

[탐색] 검색어: '핫플'
   [!] 해당 검색어에서 장소 이름을 직접 추출하지 못했습니다.

[탐색] 검색어: '감성 장소'
   [!] 해당 검색어에서 장소 이름을 직접 추출하지 못했습니다.

[탐색] 검색어: '맛집'
   [!] 해당 검색어에서 장소 이름을 직접 추출하지 못했습니다.

[탐색] 검색어: '가볼만한 곳'
   [!] 해당 검색어에서 장소 이름을 직접 추출하지 못했습니다.

[탐색] 검색어: '인스타 감성 장소'
   [!] 해당 검색어에서 장소 이름을 직접 추출하지 못했습니다.

[탐색] 검색어: '추천 장소'
   [!] 해당 검색어에서 장소 이름을 직접 추출하지 못했습니다.

[탐색] 검색어: '데이트'
   [!] 해당 검색어에서 장소 이름을 직접 추출하지 못했습니다.

[탐색] 검색어: '느좋 장소'
   [!] 해당 검색어에서 장소 이름을 직접 추출하지 못했습니다.

[결과] 총 0개의 대표 선수를 발굴했습니다.

[!] 대표 선수를 찾지 못했습니다. 검색어를 변경하거나, 다른 전략을 시도해보세요.


In [2]:
import os
import sys
import numpy as np
import pandas as pd
import geopandas as gpd
import folium
from shapely.geometry import Point
from folium.plugins import MarkerCluster # 많은 포인트를 표시하기 위해 추가

# --- 상수 ---
PLACES_CSV = "dobong_places_classified.csv" # [입력 1] 스크립트 1의 결과물
LOW20 = "low20_grids.geojson"             # [입력 2] 스크립트 2의 격자
LOW50 = "low50_grids.geojson"             # [입력 3] 스크립트 2의 격자
OUTPUT_CSV = "dobong_places_in_low_grids.csv" # [출력 1] 필터링된 장소 목록
OUTPUT_MAP = "dobong_places_in_low_grids.html" # [출력 2] 결과 지도

# 전역 (로드 후 채움)
L20 = None   # GeoDataFrame: 하위 20%
L50 = None   # GeoDataFrame: 하위 50%
BANDS = None # L20 ∪ L50 (중복 제거)

# --- 1. 격자 데이터 로드 (스크립트 2에서 가져옴) ---
def load_layers():
    """low20/low50 GeoJSON을 로드하고 BANDS를 생성합니다."""
    global L20, L50, BANDS

    if not (os.path.exists(LOW20) and os.path.exists(LOW50)):
        print(f"[오류] 필요 파일이 없습니다: {LOW20}, {LOW50}")
        return False

    l20 = gpd.read_file(LOW20).to_crs(4326).copy()
    l50 = gpd.read_file(LOW50).to_crs(4326).copy()

    # 필수 컬럼 확인
    need = {"grid_id", "final_score", "geometry"}
    if not need.issubset(l20.columns) or not need.issubset(l50.columns):
        print("[오류] GeoJSON에 grid_id, final_score, geometry 컬럼이 필요합니다.")
        return False

    # percentile 보강
    if "percentile" not in l20.columns:
        l20["percentile"] = (l20.get("rank_pct", np.nan) * 100).round(1)
    if "percentile" not in l50.columns:
        l50["percentile"] = (l50.get("rank_pct", np.nan) * 100).round(1)

    # 레이어 라벨
    l20 = l20.assign(band_label="20% 이내")
    l50 = l50.assign(band_label="50% 이내")

    # 전역 저장
    L20 = l20[["grid_id","final_score","percentile","band_label","geometry"]].copy()
    L50 = l50[["grid_id","final_score","percentile","band_label","geometry"]].copy()

    # 유니온 (중복 grid_id 제거)
    # 50% 격자와 20% 격자를 합칠 때, 20%에 해당하는 grid_id는 20% 라벨을 유지하도록 정렬 후 중복 제거
    BANDS = pd.concat([L50, L20], ignore_index=True)\
              .sort_values("band_label")\
              .drop_duplicates(subset=["grid_id"], keep="first")\
              .reset_index(drop=True)
              
    print(f"[i] 격자 로드 완료: 20% ({len(L20)}개), 50% ({len(L50)}개), 전체 BANDS ({len(BANDS)}개)")
    return True

# --- 2. 장소 데이터 로드 (신규 추가) ---
def load_places(csv_file: str):
    """스크립트 1의 CSV 결과물을 GeoDataFrame으로 로드합니다."""
    if not os.path.exists(csv_file):
        print(f"[오류] 장소 파일이 없습니다: {csv_file}")
        return None
        
    try:
        df = pd.read_csv(csv_file)
        if not ("lat" in df.columns and "lon" in df.columns):
            print(f"[오류] {csv_file}에 'lat', 'lon' 컬럼이 없습니다.")
            return None
            
        # DataFrame을 GeoDataFrame으로 변환
        geometry = [Point(xy) for xy in zip(df['lon'], df['lat'])]
        gdf_places = gpd.GeoDataFrame(df, geometry=geometry, crs="EPSG:4326")
        
        print(f"[i] 장소 로드 완료: 총 {len(gdf_places)}곳")
        return gdf_places
        
    except Exception as e:
        print(f"[오류] {csv_file} 로드 중 에러: {e}")
        return None

# --- 3. 공간 조인 (필터링) (신규 추가) ---
def filter_places_by_grids(gdf_places: gpd.GeoDataFrame, gdf_grids: gpd.GeoDataFrame):
    """
    장소(Points)와 격자(Polygons)를 공간 조인(inner)하여
    격자 내부에 포함되는 장소만 필터링합니다.
    """
    print(f"[i] 공간 조인(inner) 시작: 장소 {len(gdf_places)}곳, 격자 {len(gdf_grids)}곳")
    
    # how="inner": 양쪽에서 겹치는(intersects) 데이터만 유지
    joined_gdf = gpd.sjoin(gdf_places, gdf_grids, how="inner", predicate="intersects")
    
    # sjoin 결과 정리
    joined_gdf = joined_gdf.drop(columns="index_right", errors="ignore")
    
    # 만약 장소가 두 격자 경계에 걸쳐 2번 잡히는 경우, 더 상위(20%) 라벨을 유지
    joined_gdf = joined_gdf.sort_values("band_label")\
                           .drop_duplicates(subset=["name", "lat", "lon"], keep="first")
    
    print(f"[✔] 공간 조인 완료: 저득점 격자 내 장소 {len(joined_gdf)}곳 발견")
    return joined_gdf

# --- 4. 결과 지도 시각화 (스크립트 2 수정) ---
def create_results_map(gdf_results: gpd.GeoDataFrame) -> folium.Map:
    """필터링된 모든 장소와 격자 레이어를 Folium 지도로 생성합니다."""
    
    # 지도 중심 설정
    if len(L50) > 0:
        minx, miny, maxx, maxy = L50.total_bounds
        center = [(miny + maxy) / 2, (minx + maxx) / 2]
    else:
        center = [37.6688, 127.047] # 도봉구청
        
    m = folium.Map(location=center, zoom_start=13, tiles="cartodbpositron")

    # --- 격자 레이어 (스크립트 2와 동일) ---
    # 하위 50% (연한색)
    folium.GeoJson(
        L50.__geo_interface__,
        name="하위 50% (연한 주황)",
        style_function=lambda f: {"fillColor":"#ffedcc","color":"#b3b3b3","weight":0.7,"fillOpacity":0.35},
        tooltip=folium.GeoJsonTooltip(fields=["grid_id","final_score","percentile"],
                                      aliases=["Grid ID","Final","Percentile(%)"], localize=True)
    ).add_to(m)

    # 하위 20% (진한 빨강)
    folium.GeoJson(
        L20.__geo_interface__,
        name="하위 20% (붉은색)",
        style_function=lambda f: {"fillColor":"#ff0000","color":"#6b0000","weight":0.8,"fillOpacity":0.45},
        tooltip=folium.GeoJsonTooltip(fields=["grid_id","final_score","percentile"],
                                      aliases=["Grid ID","Final","Percentile(%)"], localize=True)
    ).add_to(m)

    # --- 필터링된 장소 포인트 추가 (신규) ---
    # 마커 클러스터 (포인트가 많을 경우 대비)
    mc_20 = MarkerCluster(name="발견 장소 (20% 이내)")
    mc_50 = MarkerCluster(name="발견 장소 (50% 이내)")
    
    color_map = {"20% 이내": "red", "50% 이내": "orange"}
    icon_map = {"20% 이내": "star", "50% 이내": "info-sign"} # 아이콘 구분
    
    for row in gdf_results.itertuples():
        band = row.band_label
        color = color_map.get(band, "blue")
        icon = icon_map.get(band, "question-sign")
        
        popup_html = f"""
        <b>{row.name}</b><br>
        <hr style='margin: 3px 0;'>
        분류: {row.top_category} > {row.sub_category}<br>
        격자: <b>{band}</b> (Grid: {row.grid_id})<br>
        점수: {row.final_score:.3f} (Pctl: {row.percentile}%)
        """
        popup = folium.Popup(popup_html, max_width=300)
        
        marker = folium.Marker(
            location=[row.lat, row.lon],
            popup=popup,
            tooltip=f"{row.name} ({band})",
            icon=folium.Icon(color=color, icon=icon, prefix='glyphicon')
        )
        
        if band == "20% 이내":
            marker.add_to(mc_20)
        else:
            marker.add_to(mc_50)

    mc_20.add_to(m)
    mc_50.add_to(m)
    
    folium.LayerControl(collapsed=False).add_to(m)
    
    # 전체 저득점 영역 기준으로 맞춤
    minx, miny, maxx, maxy = L50.total_bounds
    m.fit_bounds([[miny, minx], [maxy, maxx]])
    
    return m

def _display_map(m: folium.Map):
    """주피터/Lab이면 지도를 바로 띄움."""
    try:
        from IPython.display import display # noqa
        print(f"[✔] Jupyter 환경 감지. 지도를 바로 표시합니다.")
        display(m)
    except Exception:
        # Jupyter가 아니면 HTML 파일로 저장
        m.save(OUTPUT_MAP)
        print(f"[✔] 지도 파일 저장 완료: {OUTPUT_MAP}")
        print("(참고) 현재 환경에서는 지도 미리보기를 바로 띄울 수 없습니다. Jupyter/Lab 권장.")


# --- 메인 실행 ---
def main():
    print("=== '느좋'/'핫플' 장소 + 저득점 격자 필터링 시작 ===")
    
    # 1. 격자 로드
    print("\n--- 1. 격자 레이어 로드 중 ---")
    if not load_layers():
        sys.exit(1) # 격자 파일 없으면 종료

    # 2. 장소 로드
    print("\n--- 2. 장소 CSV 로드 중 ---")
    gdf_places = load_places(PLACES_CSV)
    if gdf_places is None:
        sys.exit(1) # 장소 파일 없으면 종료

    # 3. 공간 조인(필터링)
    print("\n--- 3. 격자 내 장소 필터링(공간 조인) 중 ---")
    filtered_results = filter_places_by_grids(gdf_places, BANDS)

    if filtered_results.empty:
        print("\n[!] 저득점 격자 내에서 발견된 장소가 없습니다.")
        return

    # 4. CSV로 저장 (geometry 컬럼 제외)
    print(f"\n--- 4. 필터링 결과 CSV 저장 중 ---")
    try:
        # CSV 저장을 위해 geometry 컬럼은 제외
        df_to_save = pd.DataFrame(filtered_results.drop(columns='geometry'))
        df_to_save.to_csv(OUTPUT_CSV, index=False, encoding="utf-8-sig")
        print(f"[✔] 결과 저장 완료: {OUTPUT_CSV} (총 {len(df_to_save)}곳)")
    except Exception as e:
        print(f"[!] CSV 저장 실패: {e}")

    # 5. 결과 요약 출력
    print("\n--- 5. 결과 요약 ---")
    print("▶ Band(격자)별 장소 수:")
    print(filtered_results['band_label'].value_counts().sort_index())
    
    print("\n▶ 상위 카테고리(top_category)별 장소 수:")
    print(filtered_results.groupby(['band_label', 'top_category']).size().unstack(fill_value=0))
    
    print("\n▶ 세부 카테고리(sub_category)별 장소 수:")
    print(pd.crosstab(filtered_results['sub_category'], filtered_results['band_label']))

    # 6. 지도 시각화
    print("\n--- 6. 결과 지도 생성 및 표시 ---")
    m = create_results_map(filtered_results)
    _display_map(m)
    
    print("\n=== 모든 작업 완료 ===")

if __name__ == "__main__":
    main()

SyntaxError: invalid non-printable character U+00A0 (2781911597.py, line 18)

In [5]:
import os
import sys
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

# --- 상수 ---
# [입력 1] 스크립트 1의 결과물
PLACES_CSV = "dobong_places_classified.csv"
# [입력 2] 스크립트 2의 격자
LOW20 = "low20_grids.geojson"
# [입력 3] 스크립트 2의 격자
LOW50 = "low50_grids.geojson"
# [출력] 필터링된 장소 목록
OUTPUT_CSV = "dobong_places_in_low_grids.csv"

# 전역 (로드 후 채움)
L20 = None   # GeoDataFrame: 하위 20%
L50 = None   # GeoDataFrame: 하위 50%
BANDS = None # L20 ∪ L50 (중복 제거)

# --- 1. 격자 데이터 로드 ---
def load_layers():
    """low20/low50 GeoJSON을 로드하고 BANDS를 생성합니다."""
    global L20, L50, BANDS

    if not (os.path.exists(LOW20) and os.path.exists(LOW50)):
        print(f"[오류] 필요 파일이 없습니다: {LOW20}, {LOW50}")
        return False

    try:
        l20 = gpd.read_file(LOW20).to_crs(4326).copy()
        l50 = gpd.read_file(LOW50).to_crs(4326).copy()
    except Exception as e:
        print(f"[오류] GeoJSON 파일 로드 실패: {e}")
        return False

    # 필수 컬럼 확인
    need = {"grid_id", "final_score", "geometry"}
    if not need.issubset(l20.columns) or not need.issubset(l50.columns):
        print("[오류] GeoJSON에 grid_id, final_score, geometry 컬럼이 필요합니다.")
        return False

    # percentile 보강
    if "percentile" not in l20.columns:
        l20["percentile"] = (l20.get("rank_pct", np.nan) * 100).round(1)
    if "percentile" not in l50.columns:
        l50["percentile"] = (l50.get("rank_pct", np.nan) * 100).round(1)

    # 레이어 라벨
    l20 = l20.assign(band_label="20% 이내")
    l50 = l50.assign(band_label="50% 이내")

    # 전역 저장
    L20 = l20[["grid_id","final_score","percentile","band_label","geometry"]].copy()
    L50 = l50[["grid_id","final_score","percentile","band_label","geometry"]].copy()

    # 유니온 (중복 grid_id 제거)
    # 50% 격자와 20% 격자를 합칠 때, 20%에 해당하는 grid_id는 20% 라벨을 유지하도록 정렬 후 중복 제거
    BANDS = pd.concat([L50, L20], ignore_index=True)\
              .sort_values("band_label")\
              .drop_duplicates(subset=["grid_id"], keep="first")\
              .reset_index(drop=True)
              
    print(f"[i] 격자 로드 완료: 20% ({len(L20)}개), 50% ({len(L50)}개), 전체 BANDS ({len(BANDS)}개)")
    return True

# --- 2. 장소 데이터 로드 ---
def load_places(csv_file: str):
    """스크립트 1의 CSV 결과물을 GeoDataFrame으로 로드합니다."""
    if not os.path.exists(csv_file):
        print(f"[오류] 장소 파일이 없습니다: {csv_file}")
        return None
        
    try:
        df = pd.read_csv(csv_file)
        if not ("lat" in df.columns and "lon" in df.columns):
            print(f"[오류] {csv_file}에 'lat', 'lon' 컬럼이 없습니다.")
            return None
            
        # DataFrame을 GeoDataFrame으로 변환
        geometry = [Point(xy) for xy in zip(df['lon'], df['lat'])]
        gdf_places = gpd.GeoDataFrame(df, geometry=geometry, crs="EPSG:4326")
        
        print(f"[i] 장소 로드 완료: 총 {len(gdf_places)}곳")
        return gdf_places
        
    except Exception as e:
        print(f"[오류] {csv_file} 로드 중 에러: {e}")
        return None

# --- 3. 공간 조인 (필터링) ---
def filter_places_by_grids(gdf_places: gpd.GeoDataFrame, gdf_grids: gpd.GeoDataFrame):
    """
    장소(Points)와 격자(Polygons)를 공간 조인(inner)하여
    격자 내부에 포함되는 장소만 필터링합니다.
    """
    print(f"[i] 공간 조인(inner) 시작: 장소 {len(gdf_places)}곳, 격자 {len(gdf_grids)}곳")
    
    # how="inner": 양쪽에서 겹치는(intersects) 데이터만 유지
    # 이것이 "범위 내에 포함되는 애들만 뽑는" 핵심 로직입니다.
    joined_gdf = gpd.sjoin(gdf_places, gdf_grids, how="inner", predicate="intersects")
    
    # sjoin 결과 정리 (오른쪽 격자의 index 컬럼 제거)
    joined_gdf = joined_gdf.drop(columns="index_right", errors="ignore")
    
    # 만약 장소가 두 격자 경계에 걸쳐 2번 잡히는 경우, 더 상위(20%) 라벨을 유지
    joined_gdf = joined_gdf.sort_values("band_label")\
                           .drop_duplicates(subset=["name", "lat", "lon"], keep="first")
    
    print(f"[✔] 공간 조인 완료: 저득점 격자 내 장소 {len(joined_gdf)}곳 발견")
    return joined_gdf

# --- 메인 실행 ---
def main():
    print("=== '느좋'/'핫플' 장소 + 저득점 격자 필터링 (CSV 저장) ===")
    
    # 1. 격자 로드
    print("\n--- 1. 격자 레이어 로드 중 ---")
    if not load_layers():
        sys.exit(1) # 격자 파일 없으면 종료

    # 2. 장소 로드
    print("\n--- 2. 장소 CSV 로드 중 ---")
    gdf_places = load_places(PLACES_CSV)
    if gdf_places is None:
        sys.exit(1) # 장소 파일 없으면 종료

    # 3. 공간 조인(필터링)
    print("\n--- 3. 격자 내 장소 필터링(공간 조인) 중 ---")
    filtered_results = filter_places_by_grids(gdf_places, BANDS)

    if filtered_results.empty:
        print("\n[!] 저득점 격자 내에서 발견된 장소가 없습니다.")
        return

    # 4. CSV로 저장 (geometry 컬럼 제외)
    print(f"\n--- 4. 필터링 결과 CSV 저장 중 ---")
    try:
        # CSV 저장을 위해 GeoDataFrame에서 geometry 컬럼은 제외
        df_to_save = pd.DataFrame(filtered_results.drop(columns='geometry'))
        
        # 컬럼 순서 정리 (원본 장소 컬럼 + 격자 정보 컬럼)
        original_cols = list(pd.read_csv(PLACES_CSV, nrows=0).columns)
        grid_cols = ["grid_id", "final_score", "percentile", "band_label"]
        final_cols = original_cols + [c for c in grid_cols if c in df_to_save.columns]
        
        # 존재하지 않는 컬럼이 있을 수 있으므로, 실제 있는 컬럼만으로 재구성
        final_cols = [c for c in final_cols if c in df_to_save.columns]
        
        df_to_save = df_to_save[final_cols]
        
        df_to_save.to_csv(OUTPUT_CSV, index=False, encoding="utf-8-sig")
        print(f"[✔] 결과 저장 완료: {OUTPUT_CSV} (총 {len(df_to_save)}곳)")
    except Exception as e:
        print(f"[!] CSV 저장 실패: {e}")

    # 5. 결과 요약 출력
    print("\n--- 5. 결과 요약 ---")
    print("▶ Band(격자)별 장소 수:")
    print(filtered_results['band_label'].value_counts().sort_index())
    
    print("\n▶ 상위 카테고리(top_category)별 장소 수:")
    print(filtered_results.groupby(['band_label', 'top_category']).size().unstack(fill_value=0))
    
    print("\n▶ 세부 카테고리(sub_category)별 장소 수:")
    print(pd.crosstab(filtered_results['sub_category'], filtered_results['band_label']))
    
    print("\n=== 모든 작업 완료 ===")

if __name__ == "__main__":
    main()

=== '느좋'/'핫플' 장소 + 저득점 격자 필터링 (CSV 저장) ===

--- 1. 격자 레이어 로드 중 ---
[i] 격자 로드 완료: 20% (32개), 50% (81개), 전체 BANDS (81개)

--- 2. 장소 CSV 로드 중 ---
[i] 장소 로드 완료: 총 146곳

--- 3. 격자 내 장소 필터링(공간 조인) 중 ---
[i] 공간 조인(inner) 시작: 장소 146곳, 격자 81곳
[✔] 공간 조인 완료: 저득점 격자 내 장소 16곳 발견

--- 4. 필터링 결과 CSV 저장 중 ---
[✔] 결과 저장 완료: dobong_places_in_low_grids.csv (총 16곳)

--- 5. 결과 요약 ---
▶ Band(격자)별 장소 수:
band_label
20% 이내     6
50% 이내    10
Name: count, dtype: int64

▶ 상위 카테고리(top_category)별 장소 수:
top_category  느좋  숨은핫플
band_label            
20% 이내         1     5
50% 이내         7     3

▶ 세부 카테고리(sub_category)별 장소 수:
band_label    20% 이내  50% 이내
sub_category                
개성있는 식당            3       3
갤러리/문화공간           1       0
독립/테마서점            1       0
둘레길/숲길             0       7
작은도서관              1       0

=== 모든 작업 완료 ===


In [6]:
import time, json, re, requests, pandas as pd

OVERPASS = "https://overpass-api.de/api/interpreter"

# 도봉구 경계 내에서 카페/베이커리/도서관/공원/정원/전망대/산책로/둘레길 + 핫플 후보군 포괄 수집
QUERY = r"""
[out:json][timeout:60];
relation["name"="도봉구"]["admin_level"="6"]["boundary"="administrative"];
map_to_area->.a;
(
  /* --- 기존 "느좋" 카테고리 --- */
  node["amenity"="cafe"](area.a);
  way["amenity"="cafe"](area.a);
  node["shop"="bakery"](area.a);
  way["shop"="bakery"](area.a);
  node["amenity"="library"](area.a);
  way["leisure"="park"](area.a);
  way["leisure"="garden"](area.a);
  node["tourism"="viewpoint"](area.a);
  node["tourism"="attraction"](area.a);
  way["highway"="footway"]["name"](area.a);      /* 이름 있는 산책로 */
  relation["route"="hiking"](area.a);           /* 둘레길/트레킹 */

  /* --- "핫플" 후보군 추가 --- */
  node["amenity"="restaurant"](area.a);
  way["amenity"="restaurant"](area.a);
  node["amenity"~"bar|pub"](area.a);
  way["amenity"~"bar|pub"](area.a);
  node["shop"="books"](area.a);
  way["shop"="books"](area.a);
  node["tourism"="gallery"](area.a);
  way["tourism"="gallery"](area.a);
  node["amenity"="arts_centre"](area.a);
  way["amenity"="arts_centre"](area.a);
);
out center tags;
"""

EXCLUDE_WORDS = [
    "스타벅스","이디야","빽다방","메가커피","더벤티",   # 프랜차이즈
    "노래방","PC방","멀티방","유흥","룸", "단란주점"   # 소음/유흥
]

# 확장된 라벨링 규칙(상위/세부)
def label_category(tags: dict, name: str):
    amenity = tags.get("amenity","")
    leisure = tags.get("leisure","")
    tourism = tags.get("tourism","")
    shop    = tags.get("shop","")
    highway = tags.get("highway","")
    route   = tags.get("route","")
    nlow = (name or "").lower()

    # 기본 타입 판정
    if amenity == "cafe" or shop == "bakery":
        base = "카페/베이커리"
    elif amenity == "library":
        base = "도서관"
    elif leisure in ("park","garden"):
        base = "공원/정원"
    elif tourism in ("viewpoint","attraction"):
        base = "전망/명소"
    elif highway == "footway" or route == "hiking":
        base = "산책로/둘레길"
    elif amenity == "restaurant":
        base = "식당"
    elif amenity in ("bar", "pub"):
        base = "바/펍"
    elif shop == "books":
        base = "서점"
    elif tourism == "gallery" or amenity == "arts_centre":
        base = "문화공간"
    else:
        base = "기타"

    # 상위 카테고리(느좋/숨은핫플) & 세부
    top, sub = "미분류","미분류"

    # --- 1. "느좋" 카테고리 분류 ---
    if base in ("공원/정원","산책로/둘레길","도서관","전망/명소"):
        top = "느좋"
        if base == "공원/정원": sub = "근린공원/정원"
        elif base == "산책로/둘레길": sub = "둘레길/숲길"
        elif base == "도서관": sub = "작은도서관"
        elif base == "전망/명소":
            if any(k in nlow for k in ["야경","노을","일몰"]): sub = "야경스팟"
            else: sub = "전망대"

    # --- 2. "카페/베이커리"는 키워드로 "느좋"과 "숨은핫플" 구분 ---
    if base == "카페/베이커리":
        # '느좋' 신호 키워드
        if any(k in nlow for k in ["티룸","전통","다도","tea","루프탑","뷰","창가","사진","포토","감성","브런치","로스터리","핸드드립","스페셜티"]):
            top, sub = "느좋","독립/감성카페"
        # '숨은핫플' 신호 키워드
        elif any(k in nlow for k in ["골목","소규모","작은","디저트","케이크","타르트", "베이킹", "클래스", "공방"]):
            top, sub = "숨은핫플","골목/디저트카페"

    # --- 3. "숨은핫플" 카테고리 분류 (신규 추가된 장소 유형) ---
    if base == "식당":
        top, sub = "숨은핫플", "개성있는 식당"
    elif base == "바/펍":
        top, sub = "숨은핫플", "특색있는 바/펍"
    elif base == "서점":
        # 이름에 '독립서점' 등의 키워드가 있으면 더 확실하지만, OSM 데이터상 보통 일반 서점과 함께 분류되므로 우선 모두 포함
        top, sub = "숨은핫플", "독립/테마서점"
    elif base == "문화공간":
        top, sub = "숨은핫플", "갤러리/문화공간"

    return base, top, sub

def fetch_overpass(q: str):
    print("[i] Overpass API에 데이터 요청 중...")
    try:
        r = requests.post(OVERPASS, data={'data': q}, timeout=90)
        r.raise_for_status()
        return r.json().get("elements", [])
    except requests.exceptions.RequestException as e:
        print(f"[!] API 요청 실패: {e}")
        return []

def main():
    print("[i] 도봉구 장소 수집 시작…")
    elems = fetch_overpass(QUERY)
    if not elems:
        print("[!] 수집된 데이터가 없습니다. 쿼리나 네트워크 상태를 확인하세요.")
        return

    rows = []
    for e in elems:
        tags = e.get("tags", {})
        name = tags.get("name")
        if not name or any(bad in name for bad in EXCLUDE_WORDS):
            continue

        lat = e.get("lat") or (e.get("center") or {}).get("lat")
        lon = e.get("lon") or (e.get("center") or {}).get("lon")
        if not (lat and lon):
            continue

        base, top, sub = label_category(tags, name)
        rows.append({
            "name": name,
            "lat": float(lat),
            "lon": float(lon),
            "base_type": base,
            "top_category": top,      # 느좋 / 숨은핫플 / 미분류
            "sub_category": sub,      # 세부 라벨
            "raw_tags": json.dumps(tags, ensure_ascii=False)
        })

    if not rows:
        print("수집 및 필터링 결과가 없습니다.")
        return

    df = pd.DataFrame(rows)

    # 근접 중복 제거 (이름 + 좌표 소수점 5자리 기준)
    df["lat_r"] = df["lat"].round(5)
    df["lon_r"] = df["lon"].round(5)
    df = df.drop_duplicates(subset=["name","lat_r","lon_r"]).drop(columns=["lat_r","lon_r"])

    # 정렬
    order = ["name","lat","lon","base_type","top_category","sub_category","raw_tags"]
    df = df[order].sort_values(["top_category","sub_category","name"]).reset_index(drop=True)

    # "미분류" 데이터는 제외하고 최종 결과물 생성
    df_filtered = df[df["top_category"] != "미분류"].reset_index(drop=True)

    if df_filtered.empty:
        print("[!] '느좋' 또는 '숨은핫플'로 분류된 장소가 없습니다.")
        return

    # --- [수정됨] "느좋" / "숨은핫플" 분리 ---
    df_neujoh = df_filtered[df_filtered["top_category"] == "느좋"].reset_index(drop=True)
    df_hotple = df_filtered[df_filtered["top_category"] == "숨은핫플"].reset_index(drop=True)

    # --- [수정됨] 파일로 각각 저장 ---
    output_neujoh = "dobong_places_neujoh.csv"
    output_hotple = "dobong_places_hotple.csv"

    if not df_neujoh.empty:
        df_neujoh.to_csv(output_neujoh, index=False, encoding="utf-8-sig")
        print(f"\n[✔] '느좋' 파일 저장 완료: {output_neujoh} (총 {len(df_neujoh)}곳)")
    else:
        print(f"\n[i] '느좋' 카테고리 장소가 없습니다.")

    if not df_hotple.empty:
        df_hotple.to_csv(output_hotple, index=False, encoding="utf-8-sig")
        print(f"\n[✔] '숨은핫플' 파일 저장 완료: {output_hotple} (총 {len(df_hotple)}곳)")
    else:
        print(f"\n[i] '숨은핫플' 카테고리 장소가 없습니다.")


    # --- [기존 유지] 전체 결과 미리보기 및 요약 ---
    print("\n--- 전체 분류 상위 10개 데이터 미리보기 (참고) ---")
    print(df_filtered.head(10).to_string(index=False))

    print("\n--- 전체 카테고리별 장소 개수 ---")
    print(df_filtered["top_category"].value_counts())

    print("\n--- 전체 세부 카테고리별 장소 개수 ---")
    print(df_filtered.groupby(["top_category", "sub_category"]).size().unstack(fill_value=0))


if __name__ == "__main__":
    main()

[i] 도봉구 장소 수집 시작…
[i] Overpass API에 데이터 요청 중...

[✔] '느좋' 파일 저장 완료: dobong_places_neujoh.csv (총 102곳)

[✔] '숨은핫플' 파일 저장 완료: dobong_places_hotple.csv (총 46곳)

--- 전체 분류 상위 10개 데이터 미리보기 (참고) ---
     name       lat        lon base_type top_category sub_category                                                                                                                                   raw_tags
I'PARK 공원 37.660194 127.045378     공원/정원           느좋      근린공원/정원                                                               {"leisure": "park", "name": "I'PARK 공원", "name:en": "I'PARK Apartment Park"}
갈대밭 어린이공원 37.674346 127.045815     공원/정원           느좋      근린공원/정원                                                                                                   {"leisure": "park", "name": "갈대밭 어린이공원"}
 개나리어린이공원 37.667345 127.037125     공원/정원           느좋      근린공원/정원                                                                                                    {"leisure": "park", "na

In [2]:
# -*- coding: utf-8 -*-
"""
Dobong-gu POI Collector (Overpass 강화판)
- nwr(node/way/relation) 전부 수집
- 산책로/둘레길 라우트 확장 (hiking/foot + named footway/path)
- name:ko 우선 사용
- 브랜드/유흥 필터를 name/brand/operator 태그로 처리
- Overpass 재시도/백오프
- 이름+좌표 반올림 중복 제거
- '느좋' / '숨은핫플' 분리 저장

필요: pip install requests pandas
"""

import time, json, random, re, requests, pandas as pd

OVERPASS = "https://overpass-api.de/api/interpreter"

# --- Overpass 쿼리 (nwr + route/foot 확장 + center 좌표) ---
QUERY = r"""
[out:json][timeout:180];
relation["name"="도봉구"]["admin_level"="6"]["boundary"="administrative"];
map_to_area->.a;

(
  /* 느좋 후보 */
  nwr["amenity"="cafe"](area.a);
  nwr["shop"="bakery"](area.a);
  nwr["amenity"="library"](area.a);
  nwr["leisure"="park"](area.a);
  nwr["leisure"="garden"](area.a);
  nwr["tourism"="viewpoint"](area.a);
  nwr["tourism"="attraction"](area.a);
  way["highway"~"footway|path"]["name"](area.a);
  relation["route"~"hiking|foot"](area.a);

  /* 핫플 후보 */
  nwr["amenity"="restaurant"](area.a);
  nwr["amenity"~"bar|pub"](area.a);
  nwr["shop"="books"](area.a);
  nwr["tourism"="gallery"](area.a);
  nwr["amenity"="arts_centre"](area.a);
  // 선택: 아래 주석 해제 가능
  // nwr["amenity"="theatre"](area.a);
  // nwr["tourism"="museum"](area.a);
);
out center tags qt;
"""

# --- 필터 단어/브랜드 ---
BRANDS = ["스타벅스", "이디야", "빽다방", "메가커피", "더벤티"]
NOISES = ["노래방", "멀티방", "단란주점", "유흥", "룸"]  # 소음/유흥

# --- 라벨링 규칙 ---
def label_category(tags: dict, name: str):
    amenity = tags.get("amenity", "")
    leisure = tags.get("leisure", "")
    tourism = tags.get("tourism", "")
    shop    = tags.get("shop", "")
    highway = tags.get("highway", "")
    route   = tags.get("route", "")
    nlow    = (name or "").lower()

    # 기본 타입 판정
    if amenity == "cafe" or shop == "bakery":
        base = "카페/베이커리"
    elif amenity == "library":
        base = "도서관"
    elif leisure in ("park", "garden"):
        base = "공원/정원"
    elif tourism in ("viewpoint", "attraction"):
        base = "전망/명소"
    elif highway in ("footway", "path") or route in ("hiking", "foot"):
        base = "산책로/둘레길"
    elif amenity == "restaurant":
        base = "식당"
    elif amenity in ("bar", "pub"):
        base = "바/펍"
    elif shop == "books":
        base = "서점"
    elif tourism == "gallery" or amenity == "arts_centre":
        base = "문화공간"
    else:
        base = "기타"

    # 상위/세부 라벨
    top, sub = "미분류", "미분류"

    # 1) 느좋
    if base in ("공원/정원", "산책로/둘레길", "도서관", "전망/명소"):
        top = "느좋"
        if base == "공원/정원":
            sub = "근린공원/정원"
        elif base == "산책로/둘레길":
            sub = "둘레길/숲길"
        elif base == "도서관":
            sub = "작은도서관"
        elif base == "전망/명소":
            sub = "야경스팟" if any(k in nlow for k in ["야경", "노을", "일몰"]) else "전망대"

    # 2) 카페/베이커리: 키워드로 느좋/핫플 분기
    if base == "카페/베이커리":
        if any(k in nlow for k in ["티룸", "전통", "다도", "tea", "루프탑", "뷰", "창가", "사진", "포토", "감성", "브런치", "로스터리", "핸드드립", "스페셜티"]):
            top, sub = "느좋", "독립/감성카페"
        elif any(k in nlow for k in ["골목", "소규모", "작은", "디저트", "케이크", "타르트", "베이킹", "클래스", "공방"]):
            top, sub = "숨은핫플", "골목/디저트카페"

    # 3) 핫플
    if base == "식당":
        top, sub = "숨은핫플", "개성있는 식당"
    elif base == "바/펍":
        top, sub = "숨은핫플", "특색있는 바/펍"
    elif base == "서점":
        top, sub = "숨은핫플", "독립/테마서점"
    elif base == "문화공간":
        top, sub = "숨은핫플", "갤러리/문화공간"

    return base, top, sub

# --- Overpass 호출 (재시도/백오프) ---
def fetch_overpass(q: str, retries: int = 3, timeout: int = 120):
    for i in range(retries):
        try:
            r = requests.post(OVERPASS, data={'data': q}, timeout=timeout)
            r.raise_for_status()
            return r.json().get("elements", [])
        except requests.exceptions.RequestException as e:
            wait = 2 * (i + 1) + random.random()
            print(f"[!] Overpass 실패({i+1}/{retries}): {e} → {wait:.1f}s 후 재시도")
            time.sleep(wait)
    return []

def main():
    print("[i] 도봉구 장소 수집 시작…")
    elems = fetch_overpass(QUERY)
    if not elems:
        print("[!] 수집된 데이터가 없습니다. 쿼리/네트워크/서버 상태 점검 요망.")
        return

    rows = []
    for e in elems:
        tags = e.get("tags", {}) or {}

        # 이름: 한국어 우선
        name = tags.get("name:ko") or tags.get("name") or tags.get("alt_name") or tags.get("official_name")
        if not name:
            continue

        # 좌표: node or center
        lat = e.get("lat") or (e.get("center") or {}).get("lat")
        lon = e.get("lon") or (e.get("center") or {}).get("lon")
        if not (lat and lon):
            continue

        # 프랜차이즈/유흥 필터 (name + brand/operator)
        brand = tags.get("brand") or tags.get("brand:ko") or ""
        operator = tags.get("operator") or tags.get("operator:ko") or ""
        if any(b in name for b in BRANDS) or any(b in brand for b in BRANDS) or any(b in operator for b in BRANDS):
            continue
        if any(n in name for n in NOISES):
            continue

        base, top, sub = label_category(tags, name)
        rows.append({
            "name": name,
            "lat": float(lat),
            "lon": float(lon),
            "base_type": base,
            "top_category": top,
            "sub_category": sub,
            "raw_tags": json.dumps(tags, ensure_ascii=False),
            "osm_type": e.get("type"),
            "osm_id": e.get("id")
        })

    if not rows:
        print("[!] 분류 가능한 데이터가 없습니다(필터 과도/OSM 밀도 낮음).")
        return

    df = pd.DataFrame(rows)

    # 근접 중복 제거 (같은 이름 + 좌표 반올림)
    df["lat_r"] = df["lat"].round(5)
    df["lon_r"] = df["lon"].round(5)
    df = df.drop_duplicates(subset=["name", "lat_r", "lon_r"]).drop(columns=["lat_r", "lon_r"])

    # 미분류는 우선 보류하지 않고 유지하려면 아래 주석 해제
    # df_filtered = df.copy()
    df_filtered = df[df["top_category"] != "미분류"].reset_index(drop=True)

    if df_filtered.empty:
        print("[!] '느좋'/'숨은핫플'로 라벨된 항목이 없습니다. 라벨 규칙을 완화해보세요.")
        return

    # 분리 저장
    df_neujoh = df_filtered[df_filtered["top_category"] == "느좋"].reset_index(drop=True)
    df_hotple = df_filtered[df_filtered["top_category"] == "숨은핫플"].reset_index(drop=True)

    out_neujoh = "dobong_places_neujoh.csv"
    out_hotple = "dobong_places_hotple.csv"

    if not df_neujoh.empty:
        df_neujoh.to_csv(out_neujoh, index=False, encoding="utf-8-sig")
        print(f"[✔] '느좋' 저장: {out_neujoh} ({len(df_neujoh)}곳)")
    else:
        print("[i] '느좋' 없음")

    if not df_hotple.empty:
        df_hotple.to_csv(out_hotple, index=False, encoding="utf-8-sig")
        print(f"[✔] '숨은핫플' 저장: {out_hotple} ({len(df_hotple)}곳)")
    else:
        print("[i] '숨은핫플' 없음")

    # 미리보기/요약
    print("\n--- 미리보기(상위 10) ---")
    print(df_filtered.head(10).to_string(index=False))

    print("\n--- 상위 카테고리 분포 ---")
    print(df_filtered["top_category"].value_counts())

    print("\n--- 세부 카테고리 분포 ---")
    print(df_filtered.groupby(["top_category", "sub_category"]).size().unstack(fill_value=0))

if __name__ == "__main__":
    main()


[i] 도봉구 장소 수집 시작…
[✔] '느좋' 저장: dobong_places_neujoh.csv (196곳)
[✔] '숨은핫플' 저장: dobong_places_hotple.csv (46곳)

--- 미리보기(상위 10) ---
       name       lat        lon base_type top_category sub_category                                                                                                                                                                                                                                              raw_tags osm_type      osm_id
        스시혼 37.646207 127.034021        식당         숨은핫플      개성있는 식당                                                                                                         {"amenity": "restaurant", "capacity": "ABOUT 40", "cuisine": "sushi", "name": "스시혼", "opening_hours": "Tue-Sun day. they have a break time."}     node  5139100541
     영즉석떡볶이 37.645924 127.031658        식당         숨은핫플      개성있는 식당                                                                                                                                  

In [5]:
# -*- coding: utf-8 -*-
"""
Dobong-gu POI Collector + Low-band Filter (All-in-One)
- Overpass 수집: nwr(node/way/relation), 산책/둘레길 확장, name:ko 우선
- 브랜드/유흥 필터 (name/brand/operator)
- 재시도/백오프
- 이름+좌표 반올림 중복 제거
- '느좋' / '숨은핫플' 분리 저장
- 저득점 격자(20%/50%) 공간조인 후 별도 CSV 저장

필요: pip install requests pandas geopandas shapely
"""

import time, json, random, requests, pandas as pd
import geopandas as gpd
from shapely.geometry import Point

# =========================
# 1) Overpass 설정/쿼리
# =========================
OVERPASS = "https://overpass-api.de/api/interpreter"

QUERY = r"""
[out:json][timeout:180];
relation["name"="도봉구"]["admin_level"="6"]["boundary"="administrative"];
map_to_area->.a;

(
  /* 느좋 후보 */
  nwr["amenity"="cafe"](area.a);
  nwr["shop"="bakery"](area.a);
  nwr["amenity"="library"](area.a);
  nwr["leisure"="park"](area.a);
  nwr["leisure"="garden"](area.a);
  nwr["tourism"="viewpoint"](area.a);
  nwr["tourism"="attraction"](area.a);
  way["highway"~"footway|path"]["name"](area.a);
  relation["route"~"hiking|foot"](area.a);

  /* 핫플 후보 */
  nwr["amenity"="restaurant"](area.a);
  nwr["amenity"~"bar|pub"](area.a);
  nwr["shop"="books"](area.a);
  nwr["tourism"="gallery"](area.a);
  nwr["amenity"="arts_centre"](area.a);
  // 선택:
  // nwr["amenity"="theatre"](area.a);
  // nwr["tourism"="museum"](area.a);
);
out center tags qt;
"""

# 프랜차이즈/유흥 제외
BRANDS = ["스타벅스", "이디야", "빽다방", "메가커피", "더벤티"]
NOISES = ["노래방", "멀티방", "단란주점", "유흥", "룸"]

# =========================
# 2) 라벨러
# =========================
def label_category(tags: dict, name: str):
    amenity = tags.get("amenity", "")
    leisure = tags.get("leisure", "")
    tourism = tags.get("tourism", "")
    shop    = tags.get("shop", "")
    highway = tags.get("highway", "")
    route   = tags.get("route", "")
    nlow    = (name or "").lower()

    if amenity == "cafe" or shop == "bakery":
        base = "카페/베이커리"
    elif amenity == "library":
        base = "도서관"
    elif leisure in ("park", "garden"):
        base = "공원/정원"
    elif tourism in ("viewpoint", "attraction"):
        base = "전망/명소"
    elif highway in ("footway", "path") or route in ("hiking", "foot"):
        base = "산책로/둘레길"
    elif amenity == "restaurant":
        base = "식당"
    elif amenity in ("bar", "pub"):
        base = "바/펍"
    elif shop == "books":
        base = "서점"
    elif tourism == "gallery" or amenity == "arts_centre":
        base = "문화공간"
    else:
        base = "기타"

    top, sub = "미분류", "미분류"

    if base in ("공원/정원", "산책로/둘레길", "도서관", "전망/명소"):
        top = "느좋"
        if base == "공원/정원":
            sub = "근린공원/정원"
        elif base == "산책로/둘레길":
            sub = "둘레길/숲길"
        elif base == "도서관":
            sub = "작은도서관"
        elif base == "전망/명소":
            sub = "야경스팟" if any(k in nlow for k in ["야경", "노을", "일몰"]) else "전망대"

    if base == "카페/베이커리":
        if any(k in nlow for k in ["티룸","전통","다도","tea","루프탑","뷰","창가","사진","포토","감성","브런치","로스터리","핸드드립","스페셜티"]):
            top, sub = "느좋", "독립/감성카페"
        elif any(k in nlow for k in ["골목","소규모","작은","디저트","케이크","타르트","베이킹","클래스","공방"]):
            top, sub = "숨은핫플", "골목/디저트카페"

    if base == "식당":
        top, sub = "숨은핫플", "개성있는 식당"
    elif base == "바/펍":
        top, sub = "숨은핫플", "특색있는 바/펍"
    elif base == "서점":
        top, sub = "숨은핫플", "독립/테마서점"
    elif base == "문화공간":
        top, sub = "숨은핫플", "갤러리/문화공간"

    return base, top, sub

# =========================
# 3) Overpass 호출 (재시도)
# =========================
def fetch_overpass(q: str, retries: int = 3, timeout: int = 120):
    for i in range(retries):
        try:
            r = requests.post(OVERPASS, data={'data': q}, timeout=timeout)
            r.raise_for_status()
            return r.json().get("elements", [])
        except requests.exceptions.RequestException as e:
            wait = 2 * (i + 1) + random.random()
            print(f"[!] Overpass 실패({i+1}/{retries}): {e} → {wait:.1f}s 후 재시도")
            time.sleep(wait)
    return []

# =========================
# 4) 저득점 격자 로더
# =========================
LOW20 = "low20_grids.geojson"
LOW50 = "low50_grids.geojson"

def load_low_bands() -> gpd.GeoDataFrame:
    l20 = gpd.read_file(LOW20).to_crs(4326).copy()
    l50 = gpd.read_file(LOW50).to_crs(4326).copy()
    need = {"grid_id", "final_score", "geometry"}
    if not need.issubset(l20.columns) or not need.issubset(l50.columns):
        raise ValueError("grid_id, final_score, geometry 컬럼 필요")
    if "percentile" not in l20.columns:
        l20["percentile"] = (l20.get("rank_pct", None) * 100).round(1)
    if "percentile" not in l50.columns:
        l50["percentile"] = (l50.get("rank_pct", None) * 100).round(1)
    l20 = l20.assign(band_label="20% 이내")[["grid_id","final_score","percentile","band_label","geometry"]]
    l50 = l50.assign(band_label="50% 이내")[["grid_id","final_score","percentile","band_label","geometry"]]
    bands = (pd.concat([l50, l20], ignore_index=True)
               .sort_values("band_label")     # 20% 이내가 먼저
               .drop_duplicates(subset=["grid_id"], keep="first")
               .reset_index(drop=True))
    return bands

# =========================
# 5) 메인
# =========================
def main():
    print("[i] 도봉구 장소 수집 시작…")
    elems = fetch_overpass(QUERY)
    if not elems:
        print("[!] 수집된 데이터가 없습니다.")
        return

    rows = []
    for e in elems:
        tags = e.get("tags", {}) or {}
        name = tags.get("name:ko") or tags.get("name") or tags.get("alt_name") or tags.get("official_name")
        if not name:
            continue

        lat = e.get("lat") or (e.get("center") or {}).get("lat")
        lon = e.get("lon") or (e.get("center") or {}).get("lon")
        if not (lat and lon):
            continue

        brand = tags.get("brand") or tags.get("brand:ko") or ""
        operator = tags.get("operator") or tags.get("operator:ko") or ""
        if any(b in name for b in BRANDS) or any(b in brand for b in BRANDS) or any(b in operator for b in BRANDS):
            continue
        if any(n in name for n in NOISES):
            continue

        base, top, sub = label_category(tags, name)
        rows.append({
            "name": name,
            "lat": float(lat),
            "lon": float(lon),
            "base_type": base,
            "top_category": top,
            "sub_category": sub,
            "raw_tags": json.dumps(tags, ensure_ascii=False),
            "osm_type": e.get("type"),
            "osm_id": e.get("id")
        })

    if not rows:
        print("[!] 분류 가능한 데이터가 없습니다.")
        return

    df = pd.DataFrame(rows)

    # 근접 중복 제거 (이름+좌표 반올림)
    df["lat_r"] = df["lat"].round(5)
    df["lon_r"] = df["lon"].round(5)
    df = df.drop_duplicates(subset=["name", "lat_r", "lon_r"]).drop(columns=["lat_r", "lon_r"])

    # '느좋'/'숨은핫플' 필터
    df_filtered = df[df["top_category"] != "미분류"].reset_index(drop=True)
    if df_filtered.empty:
        print("[!] '느좋'/'숨은핫플' 라벨 항목이 없습니다.")
        return

    # 분리 저장(전체)
    df_neujoh = df_filtered[df_filtered["top_category"] == "느좋"].reset_index(drop=True)
    df_hotple = df_filtered[df_filtered["top_category"] == "숨은핫플"].reset_index(drop=True)

    out_neujoh = "dobong_places_neujoh.csv"
    out_hotple = "dobong_places_hotple.csv"

    if not df_neujoh.empty:
        df_neujoh.to_csv(out_neujoh, index=False, encoding="utf-8-sig")
        print(f"[✔] 전체 '느좋' 저장: {out_neujoh} ({len(df_neujoh)}곳)")
    else:
        print("[i] 전체 '느좋' 없음")

    if not df_hotple.empty:
        df_hotple.to_csv(out_hotple, index=False, encoding="utf-8-sig")
        print(f"[✔] 전체 '숨은핫플' 저장: {out_hotple} ({len(df_hotple)}곳)")
    else:
        print("[i] 전체 '숨은핫플' 없음")

    # ===== 저득점 격자 공간조인 =====
    try:
        bands = load_low_bands()

        gdf_all = gpd.GeoDataFrame(
            df_filtered.copy(),
            geometry=[Point(xy) for xy in zip(df_filtered["lon"], df_filtered["lat"])],
            crs="EPSG:4326"
        )

        try:
            joined = gpd.sjoin(gdf_all, bands, how="inner", predicate="within")
        except TypeError:
            print("[warn] GeoPandas within 미지원 → intersects 사용")
            joined = gpd.sjoin(gdf_all, bands, how="inner", predicate="intersects")

        joined = (joined
                  .drop(columns="index_right", errors="ignore")
                  .sort_values("band_label")  # 20% 우선
                  .drop_duplicates(subset=["name","lat","lon"], keep="first"))

        neu_low = joined[joined["top_category"] == "느좋"].drop(columns="geometry", errors="ignore")
        hot_low = joined[joined["top_category"] == "숨은핫플"].drop(columns="geometry", errors="ignore")

        OUT_NEU_LOW = "dobong_neujoh_in_low_grids.csv"
        OUT_HOT_LOW = "dobong_hotple_in_low_grids.csv"

        base_cols = ["name","lat","lon","base_type","top_category","sub_category","raw_tags","osm_type","osm_id"]
        grid_cols = ["grid_id","final_score","percentile","band_label"]

        if not neu_low.empty:
            use_cols = [c for c in base_cols + grid_cols if c in neu_low.columns]
            neu_low[use_cols].to_csv(OUT_NEU_LOW, index=False, encoding="utf-8-sig")
            print(f"[✔] 저득점 격자 내 '느좋' 저장: {OUT_NEU_LOW} ({len(neu_low)}곳)")
        else:
            print("[i] 저득점 격자 내 '느좋' 없음")

        if not hot_low.empty:
            use_cols = [c for c in base_cols + grid_cols if c in hot_low.columns]
            hot_low[use_cols].to_csv(OUT_HOT_LOW, index=False, encoding="utf-8-sig")
            print(f"[✔] 저득점 격자 내 '숨은핫플' 저장: {OUT_HOT_LOW} ({len(hot_low)}곳)")
        else:
            print("[i] 저득점 격자 내 '숨은핫플' 없음")

    except Exception as e:
        print(f"[warn] 저득점 격자 조인 단계 건너뜀: {e}")

    # 미리보기
    print("\n--- 미리보기(상위 10) ---")
    print(df_filtered.head(10).to_string(index=False))

    print("\n--- 상위 카테고리 분포 ---")
    print(df_filtered["top_category"].value_counts())

    print("\n--- 세부 카테고리 분포 ---")
    print(df_filtered.groupby(["top_category", "sub_category"]).size().unstack(fill_value=0))

# =========================
# 6) 실행
# =========================
if __name__ == "__main__":
    try:
        main()
    except Exception as e:
        print(f"[ERROR] 실행 중 예외: {e}")


[i] 도봉구 장소 수집 시작…
[✔] 전체 '느좋' 저장: dobong_places_neujoh.csv (196곳)
[✔] 전체 '숨은핫플' 저장: dobong_places_hotple.csv (46곳)
[✔] 저득점 격자 내 '느좋' 저장: dobong_neujoh_in_low_grids.csv (9곳)
[✔] 저득점 격자 내 '숨은핫플' 저장: dobong_hotple_in_low_grids.csv (8곳)

--- 미리보기(상위 10) ---
       name       lat        lon base_type top_category sub_category                                                                                                                                                                                                                                              raw_tags osm_type      osm_id
        스시혼 37.646207 127.034021        식당         숨은핫플      개성있는 식당                                                                                                         {"amenity": "restaurant", "capacity": "ABOUT 40", "cuisine": "sushi", "name": "스시혼", "opening_hours": "Tue-Sun day. they have a break time."}     node  5139100541
     영즉석떡볶이 37.645924 127.031658        식당         숨은핫플      개성있는 식당      

#################################################################################

In [1]:
import os
import sys
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

# --- 상수 ---
# [입력 1] 스크립트 1의 '느좋' 결과물
NEUJOH_CSV = "dobong_places_neujoh.csv"
# [입력 2] 스크립트 1의 '숨은핫플' 결과물
HOTPLE_CSV = "dobong_places_hotple.csv"
# [입력 3] 스크립트 2의 격자
LOW20 = "low20_grids.geojson"
# [입력 4] 스크립트 2의 격자
LOW50 = "low50_grids.geojson"

# [출력 1] '느좋' 필터링 결과
OUTPUT_NEUJOH_CSV = "dobong_neujoh_in_low_grids.csv"
# [출력 2] '숨은핫플' 필터링 결과
OUTPUT_HOTPLE_CSV = "dobong_hotple_in_low_grids.csv"


# 전역 (로드 후 채움)
L20 = None   # GeoDataFrame: 하위 20%
L50 = None   # GeoDataFrame: 하위 50%
BANDS = None # L20 ∪ L50 (중복 제거)

# --- 1. 격자 데이터 로드 ---
def load_layers():
    """low20/low50 GeoJSON을 로드하고 BANDS를 생성합니다."""
    global L20, L50, BANDS

    if not (os.path.exists(LOW20) and os.path.exists(LOW50)):
        print(f"[오류] 필요 파일이 없습니다: {LOW20}, {LOW50}")
        return False

    try:
        l20 = gpd.read_file(LOW20).to_crs(4326).copy()
        l50 = gpd.read_file(LOW50).to_crs(4326).copy()
    except Exception as e:
        print(f"[오류] GeoJSON 파일 로드 실패: {e}")
        return False

    need = {"grid_id", "final_score", "geometry"}
    if not need.issubset(l20.columns) or not need.issubset(l50.columns):
        print("[오류] GeoJSON에 grid_id, final_score, geometry 컬럼이 필요합니다.")
        return False

    if "percentile" not in l20.columns:
        l20["percentile"] = (l20.get("rank_pct", np.nan) * 100).round(1)
    if "percentile" not in l50.columns:
        l50["percentile"] = (l50.get("rank_pct", np.nan) * 100).round(1)

    l20 = l20.assign(band_label="20% 이내")
    l50 = l50.assign(band_label="50% 이내")

    L20 = l20[["grid_id","final_score","percentile","band_label","geometry"]].copy()
    L50 = l50[["grid_id","final_score","percentile","band_label","geometry"]].copy()

    BANDS = pd.concat([L50, L20], ignore_index=True)\
              .sort_values("band_label")\
              .drop_duplicates(subset=["grid_id"], keep="first")\
              .reset_index(drop=True)
              
    print(f"[i] 격자 로드 완료: 전체 BANDS ({len(BANDS)}개)")
    return True

# --- 2. 장소 데이터 로드 ---
def load_places(csv_file: str):
    """CSV 결과물을 GeoDataFrame으로 로드합니다."""
    if not os.path.exists(csv_file):
        print(f"[오류] 장소 파일이 없습니다: {csv_file}")
        return None
        
    try:
        df = pd.read_csv(csv_file)
        if not ("lat" in df.columns and "lon" in df.columns):
            print(f"[오류] {csv_file}에 'lat', 'lon' 컬럼이 없습니다.")
            return None
            
        geometry = [Point(xy) for xy in zip(df['lon'], df['lat'])]
        gdf_places = gpd.GeoDataFrame(df, geometry=geometry, crs="EPSG:4326")
        
        print(f"[i] 장소 로드 완료: 총 {len(gdf_places)}곳")
        return gdf_places
        
    except Exception as e:
        print(f"[오류] {csv_file} 로드 중 에러: {e}")
        return None

# --- 3. 공간 조인 (필터링) ---
def filter_places_by_grids(gdf_places: gpd.GeoDataFrame, gdf_grids: gpd.GeoDataFrame):
    """장소와 격자를 공간 조인(inner)하여 격자 내 장소만 필터링합니다."""
    print(f"[i] 공간 조인(inner) 시작: 장소 {len(gdf_places)}곳, 격자 {len(gdf_grids)}곳")
    
    joined_gdf = gpd.sjoin(gdf_places, gdf_grids, how="inner", predicate="intersects")
    joined_gdf = joined_gdf.drop(columns="index_right", errors="ignore")
    
    joined_gdf = joined_gdf.sort_values("band_label")\
                           .drop_duplicates(subset=["name", "lat", "lon"], keep="first")
    
    print(f"[✔] 공간 조인 완료: 저득점 격자 내 장소 {len(joined_gdf)}곳 발견")
    return joined_gdf

# --- 4. CSV 저장 및 요약 (헬퍼 함수) ---
def save_filtered_csv(filtered_results: pd.DataFrame, original_csv: str, output_csv: str):
    """필터링된 결과를 CSV로 저장합니다."""
    if filtered_results.empty:
        print(f"\n[!] {original_csv} -> 저득점 격자 내에서 발견된 장소가 없습니다.")
        return

    try:
        df_to_save = pd.DataFrame(filtered_results.drop(columns='geometry'))
        
        # 원본 컬럼 순서 + 격자 정보
        original_cols = list(pd.read_csv(original_csv, nrows=0).columns)
        grid_cols = ["grid_id", "final_score", "percentile", "band_label"]
        final_cols = original_cols + [c for c in grid_cols if c in df_to_save.columns]
        final_cols = [c for c in final_cols if c in df_to_save.columns]
        
        df_to_save = df_to_save[final_cols]
        
        df_to_save.to_csv(output_csv, index=False, encoding="utf-8-sig")
        print(f"\n[✔] 결과 저장 완료: {output_csv} (총 {len(df_to_save)}곳)")
        
        # 요약 출력
        print("\n--- 결과 요약 ---")
        print("▶ Band(격자)별 장소 수:")
        print(filtered_results['band_label'].value_counts().sort_index())
        print("\n▶ 세부 카테고리(sub_category)별 장소 수:")
        print(pd.crosstab(filtered_results['sub_category'], filtered_results['band_label']))

    except Exception as e:
        print(f"[!] CSV 저장 실패: {e}")


# --- 메인 실행 (수정됨) ---
def main():
    print("=== '느좋'/'핫플' 장소 + 저득점 격자 필터링 (CSV 저장) ===")
    
    # 1. 격자 로드 (공통)
    print("\n--- 1. 격자 레이어 로드 중 ---")
    if not load_layers():
        sys.exit(1) # 격자 파일 없으면 종료

    # --- 2. '느좋' 파일 처리 ---
    print("\n\n=== 2. '느좋' 파일 처리 시작 ===")
    gdf_neujoh = load_places(NEUJOH_CSV)
    if gdf_neujoh is not None:
        filtered_neujoh = filter_places_by_grids(gdf_neujoh, BANDS)
        save_filtered_csv(filtered_neujoh, NEUJOH_CSV, OUTPUT_NEUJOH_CSV)
    else:
        print(f"[!] {NEUJOH_CSV} 파일을 스킵합니다.")

    # --- 3. '숨은핫플' 파일 처리 ---
    print("\n\n=== 3. '숨은핫플' 파일 처리 시작 ===")
    gdf_hotple = load_places(HOTPLE_CSV)
    if gdf_hotple is not None:
        filtered_hotple = filter_places_by_grids(gdf_hotple, BANDS)
        save_filtered_csv(filtered_hotple, HOTPLE_CSV, OUTPUT_HOTPLE_CSV)
    else:
        print(f"[!] {HOTPLE_CSV} 파일을 스킵합니다.")
    
    print("\n\n=== 모든 작업 완료 ===")

if __name__ == "__main__":
    main()

=== '느좋'/'핫플' 장소 + 저득점 격자 필터링 (CSV 저장) ===

--- 1. 격자 레이어 로드 중 ---
[i] 격자 로드 완료: 전체 BANDS (81개)


=== 2. '느좋' 파일 처리 시작 ===
[i] 장소 로드 완료: 총 102곳
[i] 공간 조인(inner) 시작: 장소 102곳, 격자 81곳
[✔] 공간 조인 완료: 저득점 격자 내 장소 9곳 발견

[✔] 결과 저장 완료: dobong_neujoh_in_low_grids.csv (총 9곳)

--- 결과 요약 ---
▶ Band(격자)별 장소 수:
band_label
20% 이내    2
50% 이내    7
Name: count, dtype: int64

▶ 세부 카테고리(sub_category)별 장소 수:
band_label    20% 이내  50% 이내
sub_category                
근린공원/정원            1       0
둘레길/숲길             0       7
작은도서관              1       0


=== 3. '숨은핫플' 파일 처리 시작 ===
[i] 장소 로드 완료: 총 46곳
[i] 공간 조인(inner) 시작: 장소 46곳, 격자 81곳
[✔] 공간 조인 완료: 저득점 격자 내 장소 8곳 발견

[✔] 결과 저장 완료: dobong_hotple_in_low_grids.csv (총 8곳)

--- 결과 요약 ---
▶ Band(격자)별 장소 수:
band_label
20% 이내    5
50% 이내    3
Name: count, dtype: int64

▶ 세부 카테고리(sub_category)별 장소 수:
band_label    20% 이내  50% 이내
sub_category                
개성있는 식당            3       3
갤러리/문화공간           1       0
독립/테마서점            1       0


=== 모든 작업 완료 ===


In [ ]:
import os
import json
import pandas as pd
from flask import Flask, request, jsonify
from openai import OpenAI

# 1. 환경 설정 및 데이터 로드
app = Flask(__name__)

# OpenAI API 클라이언트 초기화
# 환경 변수 OPENAI_API_KEY를 자동으로 사용합니다.
# 직접 입력할 경우: client = OpenAI(api_key="YOUR_API_KEY")
try:
    client = OpenAI()
except Exception as e:
    print(f"OpenAI 클라이언트 초기화 실패: {e}")
    exit()

# 장소 데이터 로드 (미리 가공된 데이터)
try:
    df_neujoh = pd.read_csv("dobong_places_neujoh.csv")
    df_hotple = pd.read_csv("dobong_places_hotple.csv")
    print(f"[i] '느좋' 장소 {len(df_neujoh)}개, '숨은핫플' 장소 {len(df_hotple)}개 로드 완료.")
except FileNotFoundError:
    print("[!] 데이터 파일(dobong_places_neujoh.csv 또는 dobong_places_hotple.csv)을 찾을 수 없습니다. 파일을 확인하세요.")
    exit()


# 2. 챗봇을 위한 Tool (함수) 정의
# 이 함수는 LLM이 호출하도록 설계되었습니다. 실제 장소 데이터를 검색하는 역할을 합니다.
def search_dobong_places(category: str = None, keyword: str = None) -> str:
    """
    도봉구 장소 데이터를 검색하고, 사용자 질문에 맞는 상위 3~5개 장소를 반환합니다.
    Args:
        category (str): 검색할 장소의 상위 카테고리 ('느좋' 또는 '숨은핫플').
        keyword (str): 장소 이름이나 세부 카테고리에 포함되어야 하는 키워드 (예: '카페', '공원', '식당').
    Returns:
        str: 검색된 장소 목록을 JSON 문자열로 반환합니다.
    """
    if category == '느좋':
        df_target = df_neujoh.copy()
    elif category == '숨은핫플':
        df_target = df_hotple.copy()
    else:
        # 카테고리가 명확하지 않으면 전체에서 검색
        df_target = pd.concat([df_neujoh, df_hotple], ignore_index=True)

    if df_target.empty:
        return json.dumps({"status": "error", "message": "해당 카테고리의 장소 데이터가 없습니다."})

    # 키워드 필터링 (장소 이름이나 세부 카테고리에서 검색)
    if keyword:
        keyword = keyword.lower()
        df_target = df_target[
            df_target['name'].str.lower().str.contains(keyword, na=False) |
            df_target['sub_category'].str.lower().str.contains(keyword, na=False)
        ]

    if df_target.empty:
        return json.dumps({"status": "not_found", "message": f"키워드 '{keyword}'에 해당하는 장소를 찾을 수 없습니다."})

    # 추천을 위해 무작위로 5개 선택 (실제 서비스에서는 평점/거리 등을 고려해야 함)
    recommendations = df_target.sample(min(5, len(df_target)))

    # LLM에 전달할 정보만 간략하게 정리
    results = recommendations[[
        'name', 
        'sub_category', 
        'top_category'
    ]].to_dict(orient='records')

    return json.dumps({"status": "success", "results": results}, ensure_ascii=False)

# 3. LLM에 정의할 함수 목록 (Tool 정의)
TOOLS = [
    {
        "type": "function",
        "function": {
            "name": "search_dobong_places",
            "description": "도봉구 장소를 '느좋' 또는 '숨은핫플' 카테고리와 키워드를 사용하여 검색합니다. 사용자에게 장소를 추천해야 할 때 이 함수를 사용하세요.",
            "parameters": {
                "type": "object",
                "properties": {
                    "category": {
                        "type": "string",
                        "enum": ["느좋", "숨은핫플"],
                        "description": "'느긋하고 좋은' 장소를 원하면 '느좋', '숨겨진 핫플레이스'를 원하면 '숨은핫플'을 사용하세요. 사용자의 질문에서 명확하게 파악할 수 없다면 이 값은 생략 가능합니다.",
                    },
                    "keyword": {
                        "type": "string",
                        "description": "검색에 사용할 핵심 키워드 (예: '카페', '공원', '데이트', '브런치').",
                    },
                },
                "required": ["keyword"],
            },
        },
    }
]

# 4. 챗봇 API 엔드포인트 구현
@app.route('/api/chatbot', methods=['POST'])
def chatbot_endpoint():
    """
    앱에서 챗봇 응답을 받기 위해 호출하는 API 엔드포인트입니다.
    요청 JSON 형식: {"messages": [{"role": "user", "content": "..."}]}
    """
    data = request.get_json()
    messages = data.get("messages", [])

    if not messages:
        return jsonify({"response": "메시지 내용을 입력해 주세요."})

    # 1. 1차 LLM 호출 (함수 호출이 필요한지 판단)
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=messages,
        tools=TOOLS,
        tool_choice="auto", # LLM이 필요하다고 판단하면 자동으로 함수 호출을 시도합니다.
        temperature=0.7,
        # 챗봇의 역할을 정의하는 시스템 프롬프트
        system="당신은 서울 도봉구의 장소 추천 전문가입니다. 사용자 질문을 분석하여 '느긋하고 좋은(느좋)' 장소 또는 '숨겨진 핫플레이스(숨은핫플)' 중 어떤 것을 원하는지 파악하고, 적절한 키워드를 추출하여 'search_dobong_places' 함수를 사용해 장소를 추천하세요. 장소 목록을 받은 후에는 친절하고 상세하게 답변해 주세요. 질문에 장소 추천이 필요 없으면 일반적인 대화를 이어가세요."
    )

    response_message = response.choices[0].message
    tool_calls = response_message.tool_calls

    # 2. 함수 호출이 필요한 경우
    if tool_calls:
        # 응답 메시지를 메시지 리스트에 추가합니다.
        messages.append(response_message)
        
        # 함수 호출 실행 및 결과 수집
        available_functions = {"search_dobong_places": search_dobong_places}
        
        for tool_call in tool_calls:
            function_name = tool_call.function.name
            function_to_call = available_functions.get(function_name)
            function_args = json.loads(tool_call.function.arguments)
            
            # 함수 실행 (장소 검색)
            function_response = function_to_call(
                category=function_args.get("category"),
                keyword=function_args.get("keyword")
            )
            
            # 함수 결과를 다시 LLM에게 전달할 메시지 리스트에 추가
            messages.append(
                {
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": function_name,
                    "content": function_response, # 검색 결과 (JSON)
                }
            )

        # 3. 2차 LLM 호출 (검색 결과를 기반으로 최종 답변 생성)
        second_response = client.chat.completions.create(
            model="gpt-4o",
            messages=messages,
            # 2차 호출 시에는 함수 호출 툴을 다시 전달할 필요는 없습니다.
        )
        return jsonify({"response": second_response.choices[0].message.content})

    # 4. 함수 호출 없이 바로 답변이 나온 경우 (일반 대화)
    return jsonify({"response": response_message.content})


# 5. 서버 실행
if __name__ == '__main__':
    # 보안 상, 실제 배포 시에는 debug=False로 설정해야 합니다.
    app.run(debug=True, port=5000)

OpenAI 클라이언트 초기화 실패: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable
[i] '느좋' 장소 102개, '숨은핫플' 장소 46개 로드 완료.
 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with stat


SystemExit: 1

c:\Users\ji sung min\Documents\project\dobong_project\.venv\lib\site-packages\IPython\core\interactiveshell.py:3587: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


: 